In [93]:
# Bibliotecas
import cx_Oracle
#cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient_21_3")
import os
import io
from PIL import Image
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import CirclePolygon
from datetime import datetime, timedelta
import numpy as np
from scipy import stats
import scipy.interpolate as interpolate
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import math
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from hmmlearn import hmm
import cv2
from statistics import median
import warnings
import base_regras_fuzzy as regras
import functions as fun
from datetime import date, timedelta
from docx import Document
from docx.shared import Inches

# Ignorar os avisos de Warnings
warnings.filterwarnings('ignore')

# Definição de Casas Decimais
casas_decimais=1

# Variável global para acumular os Resultados das Abordagens para o Relatório Final
global text
text=""

# Variável para definição de Intervalo de Janela de Tempo
intervalo=9

# Variáveis para Relatório Avaliação
coluna_abordagens=[]
coluna_seq_logica=[]
coluna_figura_merito=[]
coluna_logica_fuzzy=[]
coluna_markov=[]
coluna_vetor_ocorrencias=[]
coluna_favorabilidade_figura_Merito=[]
coluna_favorabilidade_logica_fuzzy=[]
coluna_favorabilidade_markov=[]

In [94]:
# Definição da Variável de ambiente - Acesso Oracle Database - Aplicação em Cliente Oracle
# os.environ['TNS_ADMIN']='/lib/oracle/19.6/client64/lib'
# print(os.environ.get('TNS_ADMIN'))

# Exibir (todas) variáveis de Ambiente
#!env

# Definição de Path (Gravação de arquivos)
path_2='fusao_dados/'
# Criação de diretório
if not os.path.exists(path_2):
    os.makedirs(path_2)

# Definição de Path (Gravação de Relatórios)
path_3='fusao_dados/relatorios/'
# Criação de diretório
if not os.path.exists(path_3):
    os.makedirs(path_3)

# Definição de Path (Gravação de Códigos SQL)
path_4='fusao_dados/codigosSQL/'
# Criação de diretório
if not os.path.exists(path_4):
    os.makedirs(path_4)

In [95]:
# Função para Geração de dados "Supervisionados" para validação das abordagens
def gerarDadosTestesSupervionados():
    # Leitura dos dados da Tabela Verdade - Arquivo .CSV
    # Carga dos dados em um Dataframe Pandas
    path="Tabelas_Verdade_Probabilidades_7_VAR.csv"

    # Criar um dataframe (principal) para armazenar os dados das 128 combinações - Variáveis climáticas
    df_tab_dados_sup=pd.DataFrame()
    df_tab_dados_sup=pd.read_csv(path, sep=';', usecols=['V1','V2','V3','V4','V5','V6','V7'])

    # Atribuir os valores iniciais para uso no Relatório Avaliação
    df_tabela=df_tab_dados_sup.copy()
    
    # Definição de variáveis
    #   ->umidadeRelativa
    V1_F=91
    V1_NF=0
    #   ->precipitacao
    V2_F=50
    V2_NF=0
    #   ->temperaturaMaxima
    V3_F=24
    V3_NF=37
    #   ->temperaturaMinima
    V4_F=19
    V4_NF=0
    #   ->pontoOrvalho
    V5_F=25
    V5_NF=23
    #   ->temperaturaMedCompensada
    V6_F=24
    V6_NF=29
    #   ->regra_7
    V7_F=1
    V7_NF=0

    #Troca de valores (Tabela Verdade)
        #-> F= Favorabilidade 
        #-> NF= Não Favorabilidade

    df_tab_dados_sup.loc[df_tab_dados_sup.V1==1,'V1']=V1_F
    df_tab_dados_sup.loc[df_tab_dados_sup.V1==0,'V1']=V1_NF

    df_tab_dados_sup.loc[df_tab_dados_sup.V2==1,'V2']=V2_F
    df_tab_dados_sup.loc[df_tab_dados_sup.V2==0,'V2']=V2_NF

    df_tab_dados_sup.loc[df_tab_dados_sup.V3==1,'V3']=V3_F
    df_tab_dados_sup.loc[df_tab_dados_sup.V3==0,'V3']=V3_NF

    df_tab_dados_sup.loc[df_tab_dados_sup.V4==1,'V4']=V4_F
    df_tab_dados_sup.loc[df_tab_dados_sup.V4==0,'V4']=V4_NF

    df_tab_dados_sup.loc[df_tab_dados_sup.V5==1,'V5']=V5_F
    df_tab_dados_sup.loc[df_tab_dados_sup.V5==0,'V5']=V5_NF

    df_tab_dados_sup.loc[df_tab_dados_sup.V6==1,'V6']=V6_F
    df_tab_dados_sup.loc[df_tab_dados_sup.V6==0,'V6']=V6_NF

    df_tab_dados_sup.loc[df_tab_dados_sup.V7==1,'V7']=V7_F
    df_tab_dados_sup.loc[df_tab_dados_sup.V7==0,'V7']=V7_NF

    # Renomear as colunas
    df_tab_dados_sup.rename(columns={'V1': 'umidadeRelativa','V2': 'precipitacao','V3': 'temperaturaMaxima','V4': 'temperaturaMinima',\
        'V5': 'pontoOrvalho','V6': 'temperaturaMedCompensada','V7': 'regra_7'}, inplace = True)

    # Gerar o conjunto de datas
    data_lista = pd.date_range(start ='2022-01-01', end ='2022-10-01', periods=128)   
    data_inicial = data_lista.strftime('%d/%m/%Y')

    # Adicionar novas colunas (dados complementares)
    df_tab_dados_sup['idDadosClimaticos']=464
    df_tab_dados_sup['idProjetos']=1
    df_tab_dados_sup['descricaoEstacaoClimatica']='Estação Poxoreu'
    df_tab_dados_sup['dataMedicao']=data_inicial
    df_tab_dados_sup['periodoMedicao']='safra'

    # Criar um novo dataframe para armazenar as combinações da regra 7
    df_tab_dados_regra_7=pd.DataFrame()
    df_tab_dados_regra_7['regra_7']=df_tab_dados_sup['regra_7']

    # Apagar a coluna da Regra 7 (do dataframe principal) após copiado para o novo dataframe
    df_tab_dados_sup=df_tab_dados_sup.drop(columns=['regra_7'])

    # Reorganizar as colunas na ordem dos dados originais do banco ORACLE
    df_tab_dados_sup=df_tab_dados_sup[['idDadosClimaticos','idProjetos',\
                    'descricaoEstacaoClimatica','periodoMedicao','dataMedicao','precipitacao', 'temperaturaMaxima',\
                    'temperaturaMinima','umidadeRelativa','pontoOrvalho','temperaturaMedCompensada']]                    
    
    # Exportar os dados para planilha formato .CSV - (Excel)
    df_tab_dados_sup.to_csv(path_2+"Dados_Testes_Supervisionados.csv") 

    return df_tabela, df_tab_dados_regra_7

# # Chamada de Função para Geração de dados "Supervisionados" para validação das abordagens
# df_tabela, df_tab_dados_regra_7=gerarDadosTestesSupervionados()

In [96]:
# Função para abertura de Dados de Simulação Formato .CSV
def testeDadosSimulados():
    path="Dados_Simulacao.csv"

    # Criar um dataframe (principal) para armazenar os dados das 128 combinações - Variáveis climáticas
    df_tab_dados_sup=pd.DataFrame()
    df_tab_dados_sup=pd.read_csv(path, sep=";")

    return df_tab_dados_sup

# # Chamada de Função para abertura de Dados de Simulação Formato .CSV
# df_tab_dados_sup=testeDadosSimulados()
# display(df_tab_dados_sup)

In [97]:
# Função para Execução de Teste Supervisionado
def testeSupervionado(df_tab_dados_sup, df_tab_dados_regra_7, linha):

    # -----------------------------------------TESTE SUPERVISIONADO--------------------------------------------
    #   -> Desabilitar as chamadas das funções:
    #           -> (1): Função para Entrada de Data da Janela de Tempo (def dataJanelaTempo(intervalo, data))
    #           -> (2): # Função para consulta de Dados Climáticos (Janela de Tempo) no Banco ORACLE 
    #                     (def consultarDadosClimaticos(connection))
    #           -> (3): # Função Regra 07 (def regra07(DataInicial,DataFinal))
    #           -> (4): # Função para Exibir Imagens Segmentadas participantes da Janela de Tempo
    #                     (def exibirImagensParticipantes(df_consulta_img))
    # ----------------------------------------------------------------------------------------------------------

    # Atribuição de variáveis
    df_consulta_bd=pd.DataFrame(columns=['idDadosClimaticos','idProjetos',\
                    'localEstacaoClimatica','periodoMedicao','dataMedicao','precipitacao', 'temperaturaMaxima',\
                    'temperaturaMinima','umidadeRelativa','pontoOrvalho','temperaturaMedCompensada','status'])

    # Escolha da combinação para rodar o teste
    df_consulta_bd=df_tab_dados_sup.iloc[linha:linha+1]
    regra_7=int(df_tab_dados_regra_7['regra_7'][linha])

    # Formatação de Datas para uso no código
    DataInicial="'" + df_consulta_bd['dataMedicao'].values[0] + "'"
    #DataFinal="'" + df_consulta_bd['dataMedicao'].values[1] + "'"
    DataFinal=DataInicial

    print("Data Inicial:"+DataInicial +"Data Final:"+DataFinal)

    # Verificação do valor da "Regra 7" para atribuição do flag
    if regra_7==1:
        flag_regra_7='Alta'
    if regra_7==0:
        flag_regra_7='Baixa'

    #Mostrar os resultados
    display(df_consulta_bd)
    print("Regra_7:",regra_7)
    print("Flag Regra 7:",flag_regra_7)
    print("Data Inicial:",DataInicial)
    print("Data Final:",DataFinal)

    return df_consulta_bd, regra_7, flag_regra_7, DataInicial, DataFinal, linha

# # Chamada de Função para Execução de Teste Supervisionado
# df_consulta_bd, regra_7, flag_regra_7, DataInicial, DataFinal, linha=testeSupervionado(df_tab_dados_sup, df_tab_dados_regra_7, 0)

In [98]:
# Função para Registrar Colunas de Avaliação Automática no Relatório
def registrarResultadoAvaliacao(lista_resultado_figura, lista_resultado_fuzzy, resultado_markov,\
    linha, df_tabela, vetor_ocorrencias, vetor_resultado, coluna_abordagens, coluna_seq_logica, \
    coluna_figura_merito, coluna_logica_fuzzy, coluna_markov):
    
    # Verifica se a sequência lógica do teste corresponde a sequência da Tabela Verdade
    if df_tabela['V1'].values[linha] == vetor_ocorrencias[0] and df_tabela['V2'].values[linha] == vetor_ocorrencias[1]  and \
        df_tabela['V3'].values[linha] == vetor_ocorrencias[2] and df_tabela['V4'].values[linha] == vetor_ocorrencias[3] and \
        df_tabela['V5'].values[linha] == vetor_ocorrencias[4] and df_tabela['V6'].values[linha] == vetor_ocorrencias[5] and \
        df_tabela['V7'].values[linha] == vetor_ocorrencias[6]:
            sequencia_logica=True
    else:
            sequencia_logica=False
    
    # Verificar se os resultados das três abordagens são 'Iguais"
    if lista_resultado_figura[1][1] == lista_resultado_fuzzy[1][1] and lista_resultado_figura[1][1] == resultado_markov:
        resultado_abordagens= True
    else:
        resultado_abordagens= False

    # Figura de Mérito -> Verificar se o Resultado da Abordagem está de acordo com a Referência da Tabela Verdade
                    # -> Resultados da Abordagem [Favorabilidade Baixa] - Corresponde à Referência 1
                    # -> Resultados da Abordagem [Favorabilidade Média] - Corresponde à Referência 2
                    # -> Resultados da Abordagem [Favorabilidade Alta]  - Corresponde à Referência 3
                    # -> Resposta "Acerto" para resultado igual à referência e  
                    # ->          "Erro" para resultado diferente da referência.

        # Abordagem Figura de Mérito
    if vetor_resultado['S'].values[0] == 1 and lista_resultado_figura[1][1]=="Favorabilidade Baixa":
        figura_merito="Acerto"        
        
    elif vetor_resultado['S'].values[0] == 2 and lista_resultado_figura[1][1]=="Favorabilidade Média":
        figura_merito="Acerto"        
        
    elif vetor_resultado['S'].values[0] == 3 and lista_resultado_figura[1][1]=="Favorabilidade Alta":
        figura_merito="Acerto"        
    
    else:
        figura_merito="Erro"

        # Abordagem Lógica Fuzzy
    if vetor_resultado['S'].values[0] == 1 and lista_resultado_fuzzy[1][1]=="Favorabilidade Baixa":
        logica_fuzzy="Acerto"        
                
    elif vetor_resultado['S'].values[0] == 2 and lista_resultado_fuzzy[1][1]=="Favorabilidade Média":
        logica_fuzzy="Acerto"        
                
    elif vetor_resultado['S'].values[0] == 3 and lista_resultado_fuzzy[1][1]=="Favorabilidade Alta":
        logica_fuzzy="Acerto"        
        
    else:
        logica_fuzzy="Erro"
    
        # Abordagem Cadeias Ocultas de Markov
    if vetor_resultado['S'].values[0] == 1 and resultado_markov=="Favorabilidade Baixa":
        markov="Acerto"        
        
    elif vetor_resultado['S'].values[0] == 2 and resultado_markov=="Favorabilidade Média":
        markov="Acerto"        
        
    elif vetor_resultado['S'].values[0] == 3 and resultado_markov=="Favorabilidade Alta":
        markov="Acerto"        
        
    else:
        markov="Erro"

    coluna_seq_logica.append(sequencia_logica)
    coluna_abordagens.append(resultado_abordagens)
    coluna_figura_merito.append(figura_merito)
    coluna_logica_fuzzy.append(logica_fuzzy)
    coluna_markov.append(markov)
    coluna_favorabilidade_figura_Merito.append(lista_resultado_figura[1][1])
    coluna_favorabilidade_logica_fuzzy.append(lista_resultado_fuzzy[1][1])
    coluna_favorabilidade_markov.append(resultado_markov)

    return coluna_seq_logica, coluna_abordagens, coluna_figura_merito, coluna_logica_fuzzy, coluna_markov,\
        coluna_favorabilidade_figura_Merito, coluna_favorabilidade_logica_fuzzy, coluna_favorabilidade_markov

In [99]:
# Função para Construção do Relatório para Avaliação dos Modelos 
def relatorioAvaliacaoModelos(df_tab_dados_sup, df_tabela, coluna_abordagens, coluna_seq_logica,     \
       coluna_figura_merito, coluna_logica_fuzzy, coluna_markov, coluna_favorabilidade_figura_Merito,\
       coluna_favorabilidade_logica_fuzzy, coluna_favorabilidade_markov):
    # União dos Dados Gerados para os Testes Supervisionados + Tabela Verdade (Referência)
    df_relatorio=pd.concat([df_tab_dados_sup, df_tabela], axis=1)    

    df_relatorio['Vetor_Ocorrências']=coluna_vetor_ocorrencias
    df_relatorio['Validar_Seq_Logica']=coluna_seq_logica
    df_relatorio['Resultado_Abordagens_Iguais']=coluna_abordagens
    df_relatorio['Figura_Mérito']=coluna_figura_merito
    df_relatorio['Favorabilidade_Figura_Merito']=coluna_favorabilidade_figura_Merito  
    df_relatorio['Lógica_Fuzzy']=coluna_logica_fuzzy
    df_relatorio['Favorabilidade_Lógica_Fuzzy']=coluna_favorabilidade_logica_fuzzy
    df_relatorio['Cadeias_Ocultas_Markov']=coluna_markov
    df_relatorio['Favorabilidade_Cadeias_Ocultas_Markov']=coluna_favorabilidade_markov
    df_relatorio['Pontos_Figura_Merito']=""
    df_relatorio['Pontos_Lógica_Fuzzy']=""
    df_relatorio['Pontos_Cadeias_Ocultas_Markov']=""

    # Exportar Relatório para Avaliação formato .CSV - (Excel)
    df_relatorio.to_csv(path_2+"Avaliacao_Supervisionada.csv", sep=';')

In [ ]:
# Função para Entrada de Data da Janela de Tempo
def dataJanelaTempo(intervalo, data):

    # Definição de Datas para uso na Janela de Tempo
    # Formato de data aceito: dd/mm/yyyy
    # Definição de Período da Janela de Tempo (em dias)
    periodo=timedelta(intervalo)

    # Verifica se o formato de data de entrada está correto
    #   Caso não esteja correto o formato "Digite novamente"
    while True:
        try:
            if data==0:
                input_data = input('Entre com a Data Inicial da Janela de Tempo (dd/mm/yyyy): ')
                data_inicial = datetime.strptime(input_data, '%d/%m/%Y').date()
                break
            else:
                # Opção para entrada de dados de testes (Lista de "Datas" para sequência de processamento)
                input_data = data
                data_inicial = datetime.strptime(input_data, '%d/%m/%Y').date()
                break
        except ValueError:
            print('Data em formato inválido, tente novamente!')

    # Atribuição da leitura da Data Final
    data_final=data_inicial + periodo

    # Formatar data para String (Entrada na Instrução SQL)
    DataInicial ="'" + data_inicial.strftime('%d/%m/%Y') + "'"
    DataFinal ="'" +data_final.strftime('%d/%m/%Y')+ "'"
    print("Datas Formatadas para uso em Consulta SQL (ORACLE)\n-------------------------------------------------")
    print("Data Inicial:", DataInicial, "e Data Final:", DataFinal)

    return DataInicial, DataFinal

#Chamada de Função para Entrada de Data da Janela de Tempo
#Informar a Janela de Tempo como "Intervalo"
#DataInicial, DataFinal = dataJanelaTempo(intervalo,0)

In [101]:
# Função para conexão com o Oracle Autonomous Database
def conexaoAutonomousDatabase(PYTHON_USERNAME,PYTHON_PASSWORD,PYTHON_CONNECTSTRING):
    # Objeto de Conexão
    connection=None
    try:
        connection = cx_Oracle.connect(user=PYTHON_USERNAME, password=PYTHON_PASSWORD, dsn=PYTHON_CONNECTSTRING, encoding="UTF-8") 
    
    except cx_Oracle.Error as error:
        print(error)
    
    return connection

In [ ]:
# Função para consulta de Dados Climáticos (Janela de Tempo) no Banco ORACLE 
def consultarDadosClimaticos(connection):
    # A partir do objeto de conexão executar a consulta na base de dados
    with connection.cursor() as cursor:
        try:
            sql="SELECT * FROM dados_climaticos WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial + \
                ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')"

                ### SQL do Banco Oracle
                    #SELECT * FROM "ADMIN"."dados_climaticos"
                    #WHERE DATA_MEDICAO BETWEEN TO_DATE('01/01/2000', 'DD/MM/YYYY')
                    #AND TO_DATE('11/02/2000', 'DD/MM/YYYY')

            # Carregar o resultado da consulta no DataFrame
            df_consulta_bd=pd.DataFrame(cursor.execute(sql), columns=['idDadosClimaticos','idProjetos',\
                'localEstacaoClimatica','periodoMedicao','dataMedicao','precipitacao', 'temperaturaMaxima',\
                'temperaturaMinima','umidadeRelativa','pontoOrvalho','temperaturaMedCompensada','regiaoEstacaoClimatica','status'])
            display(df_consulta_bd)

            # Contagem de linhas dataframe
            linhas_df_consulta_bd=df_consulta_bd.shape[0]

            # Verificação se o resultado da consulta ao Banco de Dados Oracle não retorna vazia
            if df_consulta_bd.empty:
                print("Consulta no período:", DataInicial, "e:", DataFinal, "não retornaram dados. Inserir outra data para processamento!!")
                raise SystemExit

            # Verificação se o resultado da consulta ao Banco de Dados Oracle retorna, no mínimo, 4 linhas (dias) de dadaos climáticos.
            #   => 4 dias são dados mínimos para o processamento dos dados na etapa de "Interpolação de Dados -> Cálculo da Spline Cúbica"
            if linhas_df_consulta_bd <4:
                print("Consulta no período:", DataInicial, "e:", DataFinal, "retornam somente " + str(linhas_df_consulta_bd) +\
                    " linhas. Dados insuficientes. \nInserir outra data para processamento!!")
                raise SystemExit
                        
    # Exceção
        except cx_Oracle.Error as e:
            error, = e.args
            print(error.message)
            print(sql)
            if (error.offset):
                print('^'.rjust(error.offset+1, ' '))

    # Fechar a Conexão
    connection.close()

    return df_consulta_bd

# Chamada de função para conexão com o Oracle Autonomous Database
connection=conexaoAutonomousDatabase(fun.PYTHON_USERNAME,fun.PYTHON_PASSWORD,fun.PYTHON_CONNECTSTRING)

# Chamada de Função para consulta de Dados Climáticos (Janela de Tempo) no Banco ORACLE
df_consulta_bd=consultarDadosClimaticos(connection)

#### Interpolação dos Dados Faltantes

In [103]:
# Função para gerar a lista de dados interpolados, de acordo com as Variáveis climáticas
def preparaDadosInterpolacao(df_consulta_bd):

    lista_interpolacao=[]
    # Eixo "Y" para Interpolação: Dados da coluna "Precipitação"
    coluna_precipitacao=df_consulta_bd['precipitacao'].to_numpy()
    lista_interpolacao.append(coluna_precipitacao)
    # Eixo "Y" para Interpolação: Dados da coluna "Temperatura Máxima"
    coluna_temperatura_maxima=df_consulta_bd['temperaturaMaxima'].to_numpy()
    lista_interpolacao.append(coluna_temperatura_maxima)
    # Eixo "Y" para Interpolação: Dados da coluna "Temperatura Mínima"
    coluna_temperatura_minima=df_consulta_bd['temperaturaMinima'].to_numpy()
    lista_interpolacao.append(coluna_temperatura_minima)
    # Eixo "Y" para Interpolação: Dados da coluna "Umidade Relativa"
    coluna_umidade_relativa=df_consulta_bd['umidadeRelativa'].to_numpy()
    lista_interpolacao.append(coluna_umidade_relativa)
    # Eixo "Y" para Interpolação: Dados da coluna "Ponto de Orvalho"
    coluna_ponto_orvalho=df_consulta_bd['pontoOrvalho'].to_numpy()
    lista_interpolacao.append(coluna_ponto_orvalho)
    # Eixo "Y" para Interpolação: Dados da coluna "Temperatura Média Compensada"
    coluna_temperatura_med_compensada=df_consulta_bd['temperaturaMedCompensada'].to_numpy()
    lista_interpolacao.append(coluna_temperatura_med_compensada)

    # Eixo "X" para Interpolação
    # Pegar a quantidade de linhas da consulta do Dataframe para definir a quantidade igual aos dados de "Y"
    X=list(range(1,df_consulta_bd['precipitacao'].shape[0]+1))

    # Gerar os Labels para o Gráfico de Interpolação (Eixo X)
    labels=[]
    for i in range(1,len(X)+1):
        labels.append("Ponto " + str(i))  
    
    return X, lista_interpolacao, labels

# Chamada de Função para gerar a lista de dados interpolados, de acordo com as Variáveis climáticas
X, lista_interpolacao, labels=preparaDadosInterpolacao(df_consulta_bd)

In [104]:
# Função para Calcular o Ajuste da Curva - Polinômios (FIT)
def calcularAjusteCurva(x,y, labels, descricao):
    # Dados de Entrada
    x = np.array(x)
    y = np.array(y)

    # Ajuste da curva a um polinômio
    p1 = np.polyfit(x,y,1)
    p2 = np.polyfit(x,y,2)
    p3 = np.polyfit(x,y,3)
    p4 = np.polyfit(x,y,4)
    p5 = np.polyfit(x,y,5)

    # Rotina para determinação do valor de R da equação da reta
    slope,intercept,r_value,p_value,std_err = stats.linregress(x,y)

    # Cálculo dos coeficientes de determinação dos polinômios de ordem 2 à 5
    yfit2 = p2[0] * pow(x,2) + p2[1] * x + p2[2]
    yresid2 = y - yfit2
    SQresid = sum(pow(yresid2,2))
    SQtotal = len(y) * np.var(y)
    R2_2 = 1 - SQresid/SQtotal

    yfit3 = p3[0] * pow(x,3) + p3[1] * pow(x,2) + p3[2] * x + p3[3]
    yresid3 = y - yfit3
    SQresid = sum(pow(yresid3,2))
    SQtotal = len(y) * np.var(y)
    R2_3 = 1 - SQresid/SQtotal

    yfit4 = p4[0] * pow(x,4) + p4[1] * pow(x,3) + p4[2] * pow(x,2) + p4[3] * x + p4[4]
    yresid4 = y - yfit4 
    SQresid = sum(pow(yresid4,2))
    SQtotal = len(y) * np.var(y) 
    R2_4 = 1 - SQresid/SQtotal

    yfit5 = p5[0] * pow(x,5) + p5[1] * pow(x,4) + p5[2] * pow(x,3) + p5[3] * pow(x,2) + p5[4] * x + p5[5]
    yresid5 = y - yfit5
    SQresid = sum(pow(yresid5,2))
    SQtotal = len(y) * np.var(y)
    R2_5 = 1 - SQresid/SQtotal

    # Impressão dos Resultados
    print('Equação da reta')
    print('Coeficientes',p1,'R2 =',pow(r_value,2))
    print('Polinômio de ordem 2')
    print('Coeficientes',p2,'R2 =',R2_2)
    print('Polinômio de ordem 3')
    print('Coeficientes',p3,'R2 =',R2_3)
    print('Polinômio de ordem 4')
    print('Coeficientes',p4,'R2 =',R2_4)
    print('Polinômio de ordem 5')
    print('Coeficientes',p5,'R2 =',R2_5)

    # Interpolação
    new_length = 50
    new_x = np.linspace(x.min(), x.max(), new_length)
    temp= interpolate.splrep(x, y, k=3)
    new_y = interpolate.splev(new_x, temp, der=0)
    new_p2=interpolate.interp1d(x,np.polyval(p2,x), kind='linear')(new_x)
    new_p3=interpolate.interp1d(x,np.polyval(p3,x), kind='linear')(new_x)
    new_p4=interpolate.interp1d(x,np.polyval(p4,x), kind='linear')(new_x)
    new_p5=interpolate.interp1d(x,np.polyval(p5,x), kind='linear')(new_x)

    #Plotagem dos gráficos - Interpolação com Spline Cúbica
    
    plt.figure(figsize=(10,10))
    plt.plot(x,y,'o')
    plt.style.use('default')

    # Plot polinômios de grau 1 até 5
    plt.subplot(2,2,1)
    plt.plot(x,y,'go')
    plt.plot(x,np.polyval(p1,x), 'g', label = 'Reta')
    plt.plot(x,np.polyval(p2,x), 'b-.', label = 'Grau 2')
    plt.plot(x,np.polyval(p3,x), label = 'Grau 3',color ='r')
    plt.plot(x,np.polyval(p4,x), label = 'Grau 4',color ='k')
    plt.plot(x,np.polyval(p5,x), 'c', label = 'Grau 5')
    plt.title("Gráfico de Dados Sem Interpolação")
    plt.xlabel("Dados Janela Temporal: "+str(descricao))
    plt.ylabel("Valores")
    plt.xticks(x, labels, rotation='vertical')
    plt.legend()

    # Plot com Interpolação e Spline Cúbica
    plt.subplot(2,2,2)
    plt.plot(x,y,'go')
    plt.plot(new_x,new_p2, 'b', label = 'Grau 2')
    plt.plot(new_x,new_p3, 'r', label = 'Grau 3')
    plt.plot(new_x,new_p4, 'k:.', label = 'Grau 4')
    plt.plot(new_x,new_p5, 'c', label = 'Grau 5')
    plt.plot(new_x, new_y,'m--', label = 'Spline Cúbica')
    plt.title("Gráfico Dados Interpolados")
    plt.xlabel("Dados Janela Temporal: " + str(descricao))
    plt.ylabel("Valores")
    plt.xticks(x, labels, rotation='vertical')
    plt.legend()
    plt.show()

    return new_x, new_y

In [ ]:
# Função para Processamento de Interpolação
def processarInterpolacao(X,labels,lista_interpolacao):
    # Criar Dataframe para armazenar os resultados das interpolações
    df_interpolar=pd.DataFrame(columns=['Y'])
    
    # Descrição das Variáveis para uso nos gráficos
    descricao_variavel=['Precipitação','Temperatura Maxima','Temperatura Mínima','Umidade Relativa',\
        'Ponto Orvalho','Temperatura Med.Compensada']
    
    # Gerar lista chave/valor (valor de cada variável/descrição)
    lista_chave_valor=list(zip(descricao_variavel,lista_interpolacao))

    # Loop para processamento das interpolações e geração dos gráficos correspondentes por variável
    for chave,valor in lista_chave_valor:
        # Chamada de Função para Calcular o Ajuste da Curva - Polinômios (FIT)
        new_x, new_y = calcularAjusteCurva(X,valor,labels,chave)
        # Salvar dados de Interpolação calculado para cada variável
        df_interpolar=df_interpolar.append({'Y':new_y},ignore_index=True)
    
    df_interpolar=df_interpolar.round(casas_decimais)
    
    return df_interpolar, np.around(new_y,casas_decimais)

# Chamada de Função para Processamento de Interpolação
df_interpolar,new_y=processarInterpolacao(X,labels,lista_interpolacao)

In [106]:
# Função para selecionar os dados interpolados para completar os dados faltantes "df_consulta_bd"
# -> DataFrame de dados Climáticos (De acordo com o intervalo definido)
def buscarValoresInterpolados(dados_interpolados, intervalo):
    dados_interpolados= list(dados_interpolados)
    vetor_dados=[]
    controle=0

    for pos in range(len(dados_interpolados)):
        if controle<pos:
            vetor_dados.append(dados_interpolados[controle])
            controle+=intervalo            
    return vetor_dados

#### Implementação das Regras - Dados Climáticos (Tabela de Regras)

In [ ]:
# Função para processamento dos valores interpolados para completar (se necessário) o conjunto de dados com os dados faltantes
def processarCompletarDadosInterpolados(df_interpolar,df_consulta_bd):

    consulta_bd_aux=[]

    #   -> Trocar o valor da coluna 'Status" para o valor "Original"
    #   -> "Original" significa que os valores NÃO são originados do "Processo de Interpolação"
    if not df_consulta_bd.empty:
        df_consulta_bd['status'] ='Original'

    # Chamada de Função para selecionar os dados interpolados para completar os dados faltantes "df_consulta_bd"
    # -> Alimentar o restante dos dados
    # -> Cada iteração alimenta uma variável ('precipitacao', 'temperaturaMaxima','temperaturaMinima','umidadeRelativa',
    #                                         'pontoOrvalho','temperaturaMedCompensada')  
    for contador in range(len(df_interpolar)):
        vetor_dados=buscarValoresInterpolados(np.hstack(df_interpolar.loc[contador].to_numpy()).round(casas_decimais), 5)
        consulta_bd_aux.append(vetor_dados)
    
    # Montar o Dataframe Auxiliar com os dados interpolados
    df_consulta_bd_aux=pd.DataFrame(np.asarray(consulta_bd_aux).T, columns=['precipitacao', 'temperaturaMaxima','temperaturaMinima',\
            'umidadeRelativa','pontoOrvalho','temperaturaMedCompensada'])
    
    # Corrigir valores negativos do cálculo de Interpolaçao
    df_consulta_bd_aux=df_consulta_bd_aux.abs()
    display(df_consulta_bd_aux)
    
    # Contagem de linhas dataframe
    linhas_df_consulta_bd=df_consulta_bd.shape[0]

    # Contagem de linhas faltantes
    linhas_faltantes=(intervalo-linhas_df_consulta_bd)+1

    # Pegar a última linha do Dataframe auxiliar nas colunas de interesse
    lista_colunas=['idDadosClimaticos','idProjetos','localEstacaoClimatica','periodoMedicao','dataMedicao','regiaoEstacaoClimatica']
    ultima_linha_df= df_consulta_bd[0:linhas_df_consulta_bd].iloc[-1]
    # print("Ultima linha:")
    # display(ultima_linha_df)
    vetor_linha_auxiliar=[]
    contador=0

    # Gerar o conjunto Auxiliar de dados Interpolados com as variáveis de interesse
    for colunas in lista_colunas:
        vetor_linha_auxiliar.append(ultima_linha_df[colunas])
        df_consulta_bd_aux.insert(contador,lista_colunas[contador],ultima_linha_df[colunas])
        contador+=1
    
    display(df_consulta_bd_aux)
    
    # Apagar as linhas "desnecessárias" do conjunto Auxiliar dados Interpolados com as variáveis de interesse
    df_consulta_bd_aux=df_consulta_bd_aux.drop(list(range(linhas_faltantes,len(df_consulta_bd_aux))))    
    display(df_consulta_bd_aux)

    # Completar o conjunto principal de dados com os dados faltantes (Interpolados)

    # Se o Conjunto Auxiliar for diferente de "vazio": 
    #   -> Significa que há dados interpolados à acrescentar no Conjunto Inicial, então:      
    #   -> Acrescentar a coluna 'Status" com o valor 'Interpolado'
    if not df_consulta_bd_aux.empty:
        df_consulta_bd_aux=df_consulta_bd_aux.assign(status ='Interpolado')
        
    # Concatenar os Dataframes (inicial) e (dados auxiliares) para complementar os dados faltantes
    df_consulta_bd=pd.concat([df_consulta_bd,df_consulta_bd_aux],ignore_index=True)
    print("Dados prontos para processamento:")
    display(df_consulta_bd)

    return df_consulta_bd

# Chamada de Função para processamento dos valores interpolados para completar (se necessário) o conjunto de dados com os dados faltantes
#df_consulta_bd=processarCompletarDadosInterpolados(df_interpolar,df_consulta_bd)

In [ ]:
#Função Regra 01 -> (Período de Molhamento Foliar)
def regra01(df_consulta_bd,casas_decimais):
   # Se período de Molhamento Foliar (umidade relativa) maior ou igual a 90%
  regra_1= df_consulta_bd[(df_consulta_bd['umidadeRelativa']>=90)]
  if not regra_1.empty:
      flag_regra_1="Alta"
      medianaRegra_01=round(regra_1['umidadeRelativa'].median(),casas_decimais)
      # Mostrar o Resultado
      print("Mediana Regra 1:",medianaRegra_01)
      print("Período Molhamento Foliar:", flag_regra_1)
      display(regra_1)

  # Se consulta (Regra Alta) retorna vazio - Verificar (Regra Média)
  if regra_1.empty:
      regra_1= df_consulta_bd[(df_consulta_bd['umidadeRelativa']>=80) & (df_consulta_bd['umidadeRelativa']<90)]
      if not regra_1.empty:
          flag_regra_1="Média"
          medianaRegra_01=round(regra_1['umidadeRelativa'].median(),casas_decimais)
          # Mostrar o Resultado
          print("Mediana Regra 1:",medianaRegra_01)
          print("Período Molhamento Foliar:", flag_regra_1)
          display(regra_1)

      # Se consulta (Regra Média) retorna vazio - Verificar (Regra Baixa)
      if regra_1.empty:
          regra_1= df_consulta_bd[(df_consulta_bd['umidadeRelativa']<80) & (df_consulta_bd['umidadeRelativa']>0)]
          if not regra_1.empty:
            flag_regra_1="Baixa"
            medianaRegra_01=round(regra_1['umidadeRelativa'].median(),casas_decimais)
            # Mostrar o Resultado
            print("Mediana Regra 1:",medianaRegra_01)
            print("Período Molhamento Foliar:", flag_regra_1)
            display(regra_1)

      ## Se consulta (Regra Baixa) retorna vazio      
      if regra_1.empty:
          medianaRegra_01=0
          flag_regra_1='Baixa'
          # Mostrar o Resultado
          print("Mediana Regra 1:", medianaRegra_01)
          print("Período Molhamento Foliar:",flag_regra_1)
          print("Não há dados da REGRA_1 para serem visualizados!!")

  return medianaRegra_01, regra_1, flag_regra_1

# Chamada de Função Regra 01 -> (Período de Molhamento Foliar)
#medianaRegra_01, regra_1, flag_regra_1=regra01(df_consulta_bd,casas_decimais)

In [ ]:
#Função Regra 02 -> (Período Mínimo de Molhamento Foliar)
def regra02(df_consulta_bd,casas_decimais):
    #  Se precipitação for maior que 6h - equivale a 1/4 de 24h
    #              (Então: Se Precipitação maior ou igual a 25% - equivale a 1/4 de 100%)
    regra_2= df_consulta_bd[(df_consulta_bd['precipitacao']>=25)]
    if not regra_2.empty:
        medianaRegra_02=round(regra_2['precipitacao'].median(),casas_decimais)
        flag_regra_2="Alta"
        # Mostrar o Resultado
        print("Mediana Regra 2:", medianaRegra_02)
        print("Período Mínimo Molhamento Foliar:", flag_regra_2)
        display(regra_2)

    # Se consulta (Regra Alta) retorna vazio - Verificar (Regra Média)
    if regra_2.empty:
        regra_2= df_consulta_bd[(df_consulta_bd['precipitacao']<25) & (df_consulta_bd['precipitacao']>=20)]
        if not regra_2.empty:
            medianaRegra_02=round(regra_2['precipitacao'].median(),casas_decimais)
            flag_regra_2="Média"
            # Mostrar o Resultado
            print("Mediana Regra 2:", medianaRegra_02)
            print("Período Mínimo Molhamento Foliar:", flag_regra_2)
            display(regra_2)

    # Se consulta (Regra Média) retorna vazio - Verificar (Regra Baixa)
    if regra_2.empty:
        regra_2= df_consulta_bd[(df_consulta_bd['precipitacao']<20) & (df_consulta_bd['precipitacao']>0) ]
        if not regra_2.empty:
            medianaRegra_02=round(regra_2['precipitacao'].median(),casas_decimais)
            flag_regra_2="Baixa"
            # Mostrar o Resultado
            print("Mediana Regra 2:", medianaRegra_02)
            print("Período Mínimo Molhamento Foliar:", flag_regra_2)
            display(regra_2)

    # Se consulta (Regra Baixa) retorna vazio
    if regra_2.empty:
        medianaRegra_02=0
        flag_regra_2='Baixa'
        # Mostrar o Resultado
        print("Mediana Regra 2:", medianaRegra_02)
        print("Período Mínimo Molhamento Foliar:",flag_regra_2)
        print("Não há dados da REGRA_2 para serem visualizados!!")   

    return medianaRegra_02, regra_2, flag_regra_2

# Chamada de Função Regra 02 -> (Período Mínimo de Molhamento Foliar)
#medianaRegra_02, regra_2, flag_regra_2=regra02(df_consulta_bd,casas_decimais)

In [ ]:
#Função Regra 03
def regra03(df_consulta_bd,casas_decimais):
    flag_df=1
    # Se a Faixa de Temperatura estiver entre 18 a 26.5°C
    regra_3= df_consulta_bd[(df_consulta_bd['temperaturaMinima']>=18) & (df_consulta_bd['temperaturaMaxima']<=26.5)]
    if not regra_3.empty:
        ## Tempertatura Inicial (Temperatura Mínima)
        medianaRegra_03_1=round(regra_3['temperaturaMinima'].median(),casas_decimais)
        ## Tempertatura Final (Temperatura Máxima)
        medianaRegra_03_2=round(regra_3['temperaturaMaxima'].median(),casas_decimais)
        flag_regra_3="Alta"
        # Mostrar o Resultado
        print("Mediana Regra 3 - Temperatura Inicial:",medianaRegra_03_1)
        print("Mediana Regra 3 - Temperatura Final:",medianaRegra_03_2)
        print("Faixa de Temperatura:", flag_regra_3)
        display(regra_3)
    
    # Se consulta (Regra Alta) retorna vazio - Verificar (Regra Média)
    if regra_3.empty:       
        regra_3= df_consulta_bd[(df_consulta_bd['temperaturaMinima']>=15.1) & (df_consulta_bd['temperaturaMaxima']<=17.9)]
        if not regra_3.empty:
            ## Tempertatura Inicial (Temperatura Mínima)
            medianaRegra_03_1=round(regra_3['temperaturaMinima'].median(),casas_decimais)
            ## Tempertatura Final (Temperatura Máxima)
            medianaRegra_03_2=round(regra_3['temperaturaMaxima'].median(),casas_decimais)
            flag_regra_3="Média"
            # Mostrar o Resultado
            print("Mediana Regra 3 - Temperatura Inicial:",medianaRegra_03_1)
            print("Mediana Regra 3 - Temperatura Final:",medianaRegra_03_2)
            print("Faixa de Temperatura:", flag_regra_3)
            display(regra_3)
    
    # Se consulta (Regra Média) retorna vazio - Verificar (Regra Baixa)
    if regra_3.empty:
        regra_3= df_consulta_bd[(df_consulta_bd['temperaturaMinima']>=0) & (df_consulta_bd['temperaturaMaxima']<=15)]   
        if not regra_3.empty:
            ## Tempertatura Inicial (Temperatura Mínima)
            medianaRegra_03_1=round(regra_3['temperaturaMinima'].median(),casas_decimais)
            ## Tempertatura Final (Temperatura Máxima)
            medianaRegra_03_2=round(regra_3['temperaturaMaxima'].median(),casas_decimais)
            flag_regra_3="Baixa"
            # Mostrar o Resultado
            print("Mediana Regra 3 - Temperatura Inicial:",medianaRegra_03_1)
            print("Mediana Regra 3 - Temperatura Final:",medianaRegra_03_2)
            print("Faixa de Temperatura:", flag_regra_3)
            display(regra_3)

    # Se consulta (Regra Baixa) retorna vazio
    if regra_3.empty:
        flag_df=0
        ## Temperatura Inicial (Temperatura Mínima)
        medianaRegra_03_1=0
        ## Temperatura Final (Temperatura Máxima)
        medianaRegra_03_2=0
        flag_regra_3="Baixa"
        regra_3=0
        # Mostrar o Resultado
        print("Mediana Regra 3 - Temperatura Inicial:",medianaRegra_03_1)
        print("Mediana Regra 3 - Temperatura Final:",medianaRegra_03_2)
        print("Faixa de Temperatura:",flag_regra_3)
        print("Não há dados da REGRA_3 para serem visualizados!!")        
        
    return medianaRegra_03_1, medianaRegra_03_2, regra_3, flag_regra_3, flag_df

#Chamada de Função Regra 03 
#medianaRegra_03_1, medianaRegra_03_2, regra_3, flag_regra_3, flag_df=regra03(df_consulta_bd,casas_decimais)

In [ ]:
#Função Regra 04
def regra04(df_consulta_bd,casas_decimais):
    # Maior Temperatura da Janela (Temperatura Máxima)
    regra_4_valor= df_consulta_bd['temperaturaMaxima'].max()
    # Flag para verificação da situação de Favorabilidade da variável "Temperatura Máxima"
    flag_regra_4=""

    # Compara se o valor da maior Temperatura Máxima dentro da faixa de favorabilidade (Baixa, Média ou Alta)
    if (regra_4_valor>=18) & (regra_4_valor<=26.5):
        flag_regra_4="Alta"
        regra_4=1

    elif ((regra_4_valor>=15.1) & (regra_4_valor<=17.9)) | ((regra_4_valor>26.5) & (regra_4_valor<=30)) :
        flag_regra_4="Média"
        regra_4=1
        
    elif ((regra_4_valor>0) & (regra_4_valor<=15)) | ((regra_4_valor>30) & (regra_4_valor<=42)):
        flag_regra_4="Baixa"
        regra_4=1

    # Temperatura de corte = 42 Graus (Erro de medição). 
    elif (regra_4_valor==0) | (regra_4_valor>42):
        flag_regra_4="Baixa"
        regra_4=0
                    
    # Mostrar o Resultado
    print("Temperatura Máxima da Janela (valor):",regra_4_valor)
    print("Valor de Favorabilidade (Regra 4):",flag_regra_4)
    print("Valor (Regra 4):",regra_4)

    return regra_4, regra_4_valor, flag_regra_4

# Chamada Função Regra 04
#regra_4, regra_4_valor, flag_regra_4=regra04(df_consulta_bd,casas_decimais)

In [ ]:
#Função Regra 05
def regra05(df_consulta_bd,casas_decimais):
    # Menor Temperatura da Janela (Temperatura Mínima)
    regra_5_valor= df_consulta_bd['temperaturaMinima'].min()
    # Compara se o valor da maior Temperatura Mínima está dentro da faixa de favorabilidade
    if (regra_5_valor>=18) & (regra_5_valor<=26.5):
        flag_regra_5="Alta"
        regra_5=1

    elif ((regra_5_valor>=15.1) & (regra_5_valor<=17.9)) | ((regra_5_valor>26.5) & (regra_5_valor<=30)) :
        flag_regra_5="Média"
        regra_5=1
        
    elif ((regra_5_valor>=0) & (regra_5_valor<=15)) | (regra_5_valor>30):
        flag_regra_5="Baixa"
        regra_5=1
    
    elif (regra_5_valor<=-1):
        flag_regra_5="Baixa"
        regra_5=0
            
    # Mostrar o Resultado
    print("Temperatura Mínima da Janela (valor):",regra_5_valor)
    print("Valor de Favorabilidade (Regra 5):",flag_regra_5)
    print("Valor (Regra 5):",regra_5)

    return regra_5, regra_5_valor, flag_regra_5

# Chamada de Função Regra 05
#regra_5, regra_5_valor, flag_regra_5=regra05(df_consulta_bd,casas_decimais)

In [ ]:
# Função Regra 06
def regra06(df_consulta_bd,casas_decimais):
    # Calcular o Ponto de Orvalho. Se diferença da Temperatura Média for menor (2 graus) que o Ponto de Orvalho
    regra_6=df_consulta_bd[(df_consulta_bd['temperaturaMedCompensada'] - (df_consulta_bd['pontoOrvalho'])) <=2]
    # Mostrar o Resultado
    if regra_6.empty:        
            ## Se o valor é vazio "NAN"
            # => quando o resultado da consulta (BD) retorna vazio
        medianaRegra_06=0
        flag_regra_6="Baixa"
        print("Mediana Regra 06:",medianaRegra_06)
        print("Valor de Favorabilidade (Regra 6):",flag_regra_6)
        print("Não há dados da REGRA_6 para serem visualizados!!")
    else:
        medianaRegra_06_PO=round(regra_6['pontoOrvalho'].median(),casas_decimais)
        medianaRegra_06_TEMP=round(regra_6['temperaturaMedCompensada'].median(),casas_decimais)
        # Diferença entre Temperatura e Ponto de Orvalho
        medianaRegra_06=abs(round(medianaRegra_06_PO - medianaRegra_06_TEMP, casas_decimais))
        flag_regra_6="Alta"
        print("Mediana Regra 06 - Ponto Orvalho:",medianaRegra_06_PO)
        print("Mediana Regra 06 - Temperatura Média Compensada:",medianaRegra_06_TEMP)
        print("Mediana Regra 06:",medianaRegra_06)
        print("Valor de Favorabilidade (Regra 6):",flag_regra_6)
        display(regra_6)

    return regra_6, medianaRegra_06, flag_regra_6

# Chamada de Função Regra 06
#regra_6, medianaRegra_06, flag_regra_6=regra06(df_consulta_bd,casas_decimais)

In [ ]:
# Função Regra 07
def regra07(DataInicial,DataFinal):
    # Verificar se a imagem tem a doença (0-> Sem a doença e 1-> Com a presença da doença)
    # Dados do Cultivar /dados de classificação da imagem

    # Chamada de função para conexão com o Oracle Autonomous Database
    connection=conexaoAutonomousDatabase(fun.PYTHON_USERNAME,fun.PYTHON_PASSWORD,fun.PYTHON_CONNECTSTRING)

    # A partir do objeto de conexão executar a consulta na base de dados
    with connection.cursor() as cursor:
        try:
            # Consulta no Banco de Dados Oracle
            sql_1="SELECT RESULTADO                                        \
                 FROM CLASSIFICACOES                                       \
                 WHERE ID_BANCO_IMAGENS = (SELECT MIN(ID_BANCO_IMAGENS)    \
                                             FROM IMAGEM_CLIMA_FAVORAB     \
                                             WHERE ID_DADOS_CLIMATICOS = (SELECT MIN(ID_DADOS_CLIMATICOS) \
                                                                          FROM DADOS_CLIMATICOS           \
                                                                          WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial +          \
                                                                            ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')))"
                                    
            # Carregar o resultado da consulta no DataFrame
            resultado_classificacao=pd.DataFrame(cursor.execute(sql_1),columns=['Resultado'])

            # Consulta no Banco de Dados Oracle
            sql_2="SELECT ID_IMG_SEGMENTADAS                             \
                 FROM IMAGENS_SEGMENTADAS                                \
                 WHERE ID_BANCO_IMAGENS = (SELECT MIN(ID_BANCO_IMAGENS)  \
                                             FROM IMAGEM_CLIMA_FAVORAB   \
                                             WHERE ID_DADOS_CLIMATICOS = (SELECT MIN(ID_DADOS_CLIMATICOS) \
                                                                          FROM DADOS_CLIMATICOS           \
                                                                          WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial +          \
                                                                            ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')))\
                ORDER BY ID_IMG_SEGMENTADAS "
                                                       
            # Carregar o resultado da consulta no DataFrame
            df_consulta_img=pd.DataFrame(cursor.execute(sql_2), columns=['idDadosClimaticos'])

            #*********************************************
            # Consultas Classificação - Exibição Dashboard
            #*********************************************

            # Consulta Relatório Classificação (Banco de Dados Oracle)
            sql_3="SELECT RELATORIO_CLASSIFICACAO                         \
                 FROM CLASSIFICACOES                                      \
                 WHERE ID_BANCO_IMAGENS = (SELECT MIN(ID_BANCO_IMAGENS)   \
                                             FROM IMAGEM_CLIMA_FAVORAB    \
                                             WHERE ID_DADOS_CLIMATICOS = (SELECT MIN(ID_DADOS_CLIMATICOS) \
                                                                          FROM DADOS_CLIMATICOS           \
                                                                          WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial +          \
                                                                            ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')))"
            # Executar comando SQL
            cursor.execute(sql_3)
                                 
            # Recebe Dados tipo CLOB do Banco ORACLE
            for row in cursor:
                df_consulta_relat_classificacao=pd.DataFrame([[row[0].read()]], columns=['relatorioClassificacao'])

            # Consulta Matriz de Confusão (Banco de Dados Oracle)
            sql_4="SELECT MATRIZ_CONFUSAO                                 \
                 FROM CLASSIFICACOES                                      \
                 WHERE ID_BANCO_IMAGENS = (SELECT MIN(ID_BANCO_IMAGENS)   \
                                             FROM IMAGEM_CLIMA_FAVORAB    \
                                             WHERE ID_DADOS_CLIMATICOS = (SELECT MIN(ID_DADOS_CLIMATICOS) \
                                                                          FROM DADOS_CLIMATICOS           \
                                                                          WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial +          \
                                                                            ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')))"
            # Executar o código SQL
            cursor.execute(sql_4)

            # Recebe imagem BLOB do Banco ORACLE
            data_ref_1 = cursor.fetchall()
            image_ref_1 = data_ref_1[0][0].read()
                        
            # Converte BLOB em bytes e array para plotar a imagem (Matplotlib)
            image_ref_1 = np.asarray(Image.open(io.BytesIO(image_ref_1)))
            
            # Consulta Relatório Classificação (Banco de Dados Oracle)
            sql_5="SELECT CURVA_ROC                                       \
                 FROM CLASSIFICACOES                                      \
                 WHERE ID_BANCO_IMAGENS = (SELECT MIN(ID_BANCO_IMAGENS)   \
                                             FROM IMAGEM_CLIMA_FAVORAB    \
                                             WHERE ID_DADOS_CLIMATICOS = (SELECT MIN(ID_DADOS_CLIMATICOS) \
                                                                          FROM DADOS_CLIMATICOS           \
                                                                          WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial +          \
                                                                            ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')))"
            # Executar o código SQL
            cursor.execute(sql_5)

            # Recebe imagem BLOB do Banco ORACLE
            data_ref_2 = cursor.fetchall()
            image_ref_2 = data_ref_2[0][0].read()
                        
            # Converte BLOB em bytes e array para plotar a imagem (Matplotlib)
            image_ref_2 = np.asarray(Image.open(io.BytesIO(image_ref_2)))

            # Consulta no Banco de Dados Oracle (Instâncias duplicadas)
            sql_6="SELECT TUPLAS_BINARIAS_LIMPAS                           \
                 FROM CLASSIFICACOES                                       \
                 WHERE ID_BANCO_IMAGENS = (SELECT MIN(ID_BANCO_IMAGENS)    \
                                             FROM IMAGEM_CLIMA_FAVORAB     \
                                             WHERE ID_DADOS_CLIMATICOS = (SELECT MIN(ID_DADOS_CLIMATICOS) \
                                                                          FROM DADOS_CLIMATICOS           \
                                                                          WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial +          \
                                                                            ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')))"
            
            # Carregar o resultado da consulta no DataFrame
            df_instancias_duplicadas=pd.DataFrame(cursor.execute(sql_6),columns=['TUPLAS BINARIAS LIMPAS'])
            
            # Consulta no Banco de Dados Oracle (ID cadastro da relação imagem, dados climáticos e favorabilidade)
            #   => uso nas recomendações para indicar os dados de plantas_soja 
            sql_7="SELECT MIN(ID_DADOS_PLANTA_SOJA) FROM IMAGEM_CLIMA_FAVORAB WHERE ID_BANCO_IMAGENS =    \
                                            (SELECT MIN(ID_BANCO_IMAGENS) FROM IMAGEM_CLIMA_FAVORAB       \
                                             WHERE ID_DADOS_CLIMATICOS = (SELECT MIN(ID_DADOS_CLIMATICOS) \
                                                                          FROM DADOS_CLIMATICOS           \
                                                                          WHERE DATA_MEDICAO BETWEEN TO_DATE(" + DataInicial +          \
                                                                            ",'DD/MM/YYYY') and TO_DATE("+ DataFinal + ",'DD/MM/YYYY')))"
                                    
            # Carregar o resultado da consulta no DataFrame
            df_id_imagem_clima_favorab=pd.DataFrame(cursor.execute(sql_7),columns=['ID_IMAGEM_CLIMA_FAVORAB'])

    # Exceção
        except cx_Oracle.Error as e:
            error, = e.args
            print(error.message)
            print(sql_2)
            if (error.offset):
                print('^'.rjust(error.offset+1, ' '))

    # Fechar a Conexão
    connection.close()

    # Carregar a Imagem Segmentada de Referência no "subplot" (posição 1)
    fig_matriz_curva_classificacao = plt.figure(figsize = (25,25))
    plt.subplot(1, 5, 1)
    plt.axis('off')
    plt.imshow(image_ref_1)

    #------------------------------
    # Carregar a Imagem Canal Verde no "subplot" (posição 2)
    plt.subplot(1, 5, 2)
    plt.axis('off')
    plt.imshow(image_ref_2)

    if not resultado_classificacao.empty:
        regra_7=int(resultado_classificacao['Resultado'])

        if int(resultado_classificacao['Resultado'])==1:
            flag_regra_7="Alta"
        elif int(resultado_classificacao['Resultado'])==0:
            flag_regra_7="Baixa"
        
        # Mostrar o Resultado
        print("Valor de Favorabilidade (Regra 7):",int(resultado_classificacao['Resultado']))
        print("Valor de Favorabilidade (Flag):",flag_regra_7)

        print("\nImagens Segmentadas que participam da Classificação:")
        for ind in df_consulta_img.index:
            print("IMAGEM SEGMENTADA:",df_consulta_img['idDadosClimaticos'][ind])
        
        return regra_7, df_consulta_img, flag_regra_7, df_consulta_relat_classificacao, fig_matriz_curva_classificacao, df_instancias_duplicadas, df_id_imagem_clima_favorab

    else:
        print("Resultado da Classificação da Imagem retornou vazio. Verifique o Cadastro!!")
        return 0,0,0
    
# Chamada de Função Regra 07
#regra_7, df_consulta_img, flag_regra_7, df_consulta_relat_classificacao, \
#        fig_matriz_curva_classificacao,df_instancias_duplicadas,df_id_imagem_clima_favorab=regra07(DataInicial,DataFinal)

In [ ]:
# Função para Exibir Imagens Segmentadas participantes da Janela de Tempo
def exibirImagensParticipantes(df_consulta_img):
    
    # Ordenar na sequência as imagens: em "Verde", "Amarela" e "Marrom"
    #   -> para a exibição no Matplotlib
    df_consulta_img= df_consulta_img.sort_values(['idDadosClimaticos'])
    plt.figure(figsize=[20,20])
    n=2

    # Chamada de função para conexão com o Oracle Autonomous Database
    connection=conexaoAutonomousDatabase(fun.PYTHON_USERNAME,fun.PYTHON_PASSWORD,fun.PYTHON_CONNECTSTRING)

    # A partir do objeto de conexão executar as consultas na base de dados ORACLE
    with connection.cursor() as cursor:
        try:
            sql_3="SELECT IMG_SEGMENT_REFERENCIA FROM IMAGENS_SEGMENTADAS WHERE ID_IMG_SEGMENTADAS =" + str(df_consulta_img['idDadosClimaticos'][0])
            cursor.execute(sql_3)

            # Recebe imagem BLOB do Banco ORACLE
            data_ref = cursor.fetchall()
            image_ref = data_ref[0][0].read()
                        
            # Converte BLOB em bytes e array para plotar a imagem (Matplotlib)
            image_ref = np.asarray(Image.open(io.BytesIO(image_ref)))

        # Exceção
        except cx_Oracle.Error as e:
            error, = e.args
            print(error.message)
            print(sql_3)
            if (error.offset):
                print('^'.rjust(error.offset+1, ' '))

        # Carregar a Imagem Segmentada de Referência no "subplot" (posição 1)
        plt.subplot(1, 4, 1)
        plt.imshow(image_ref)
        plt.title("Imagem Segmentada Referência", fontsize=10)

        # Loop para carregar as imagens segmentadas (Classes: Verde, Amarela e Marrom)
        for ind in df_consulta_img.index:
                                   
                try:
                    sql_3="SELECT IMG_ROTULO_ESCOLHIDO FROM IMAGENS_SEGMENTADAS WHERE ID_IMG_SEGMENTADAS =" + str(df_consulta_img['idDadosClimaticos'][ind])                
                    cursor.execute(sql_3)

                    # Recebe imagem BLOB do Banco ORACLE
                    data = cursor.fetchall()
                    image = data[0][0].read()
                                
                    # Converte BLOB em bytes e array para plotar a imagem (Matplotlib)
                    image = np.asarray(Image.open(io.BytesIO(image)))

                    sql_4="SELECT CLASSE_COR FROM IMAGENS_SEGMENTADAS WHERE ID_IMG_SEGMENTADAS =" + str(df_consulta_img['idDadosClimaticos'][ind])                
                    cor = cursor.execute(sql_4).fetchall()
                    
                    # Carregar as Imagens Segmentadas no "subplot" (posições 2,3 e 4)                         
                    plt.subplot(1, 4, n)
                    plt.imshow(image)
                    plt.title("Imagem Segmentada:" + str(df_consulta_img['idDadosClimaticos'][ind]) + \
                        " - (Cor):"+ str(cor).strip("[(',)]"), fontsize=10)
                    n+=1
                
            # Exceção
                except cx_Oracle.Error as e:
                    error, = e.args
                    print(error.message)
                    print(sql_3)
                    if (error.offset):
                        print('^'.rjust(error.offset+1, ' '))

    # Fechar a Conexão
    connection.close()
    
#Chamada de Função para Exibir Imagens Segmentadas participantes da Janela de Tempo
#exibirImagensParticipantes(df_consulta_img)

In [ ]:
# Função para Contagem de Ocorrências por Regras
def contagemOcorrencias(df_consulta_bd,regra_1,regra_2,regra_3,regra_4,regra_5,regra_6,regra_7, \
      medianaRegra_01,medianaRegra_02,regra_4_valor,regra_5_valor,medianaRegra_06, casas_decimais, flag_df):

   totalJanela=len(df_consulta_bd.index)
   print("Total de Dados (leituras diárias) da Janela-> "+ str(totalJanela))
   ocorrenciasRegra_1= len(regra_1.index)
   print("Ocorrências Regra 1-> "+ str(ocorrenciasRegra_1))
   ocorrenciasRegra_2= len(regra_2.index)
   print("Ocorrências Regra 2-> "+ str(ocorrenciasRegra_2))
   
   # Se Regra 3 retornar "0" ler o valor normalmente
   #  -> Se Diferente de "0" ler a quantidade de linhas do Dataframe Pandas
   if flag_df==0:
      ocorrenciasRegra_3=regra_3
   else:
      ocorrenciasRegra_3= len(regra_3.index)
   
   print("Ocorrências Regra 3-> "+ str(ocorrenciasRegra_3))
   ocorrenciasRegra_4= regra_4
   print("Ocorrências Regra 4-> "+ str(ocorrenciasRegra_4))
   ocorrenciasRegra_5= regra_5
   print("Ocorrências Regra 5-> "+ str(ocorrenciasRegra_5))
   ocorrenciasRegra_6= len(regra_6.index)
   print("Ocorrências Regra 6-> "+ str(ocorrenciasRegra_6))
   ocorrenciasRegra_7= regra_7
   print("Ocorrências Regra 7-> "+ str(ocorrenciasRegra_7))

   # Vetor de Valores das Regras (Valores de Referência)      
   dados_regras=np.array([medianaRegra_01,medianaRegra_02,medianaRegra_03_1, medianaRegra_03_2,regra_4_valor,regra_5_valor,medianaRegra_06, regra_7])

   return dados_regras, totalJanela, ocorrenciasRegra_1, ocorrenciasRegra_2, ocorrenciasRegra_3, ocorrenciasRegra_4,\
      ocorrenciasRegra_5, ocorrenciasRegra_6, ocorrenciasRegra_7

# Função para Contagem de Ocorrências por Regras
#dados_regras, totalJanela, ocorrenciasRegra_1, ocorrenciasRegra_2, ocorrenciasRegra_3, ocorrenciasRegra_4,                   \
#   ocorrenciasRegra_5, ocorrenciasRegra_6, ocorrenciasRegra_7=contagemOcorrencias(df_consulta_bd,regra_1,regra_2,regra_3,    \
#   regra_4,regra_5,regra_6,regra_7,medianaRegra_01,medianaRegra_02,regra_4_valor,regra_5_valor,medianaRegra_06, casas_decimais, flag_df)

In [ ]:
# Função para Plotar gráfico de ocorrências
def plotarOcorrencias(ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,ocorrenciasRegra_4,ocorrenciasRegra_5,\
   ocorrenciasRegra_6,ocorrenciasRegra_7):
   x=[1,2,3,4,5,6,7]
   y=[ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,ocorrenciasRegra_4,ocorrenciasRegra_5,ocorrenciasRegra_6,
      ocorrenciasRegra_7]
   labels = ['Regra 1','Regra 2','Regra 3','Regra 4','Regra 5','Regra 6','Regra 7']
   plt.plot(x,y,'k--')
   plt.plot(x,y,'go')
   plt.grid(True)
   plt.title("Gráfico de Ocorrências vs Favorabildade")
   plt.xlabel("Base de Regras de Favorabilidade da FAS")
   plt.ylabel("Ocorrências")
   plt.show()

   return x,y

# Chamada de Função para Plotar gráfico de ocorrências
#x,y=plotarOcorrencias(ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,ocorrenciasRegra_4,ocorrenciasRegra_5,\
#   ocorrenciasRegra_6,ocorrenciasRegra_7)

In [ ]:
# Função para Plotar gráfico de ocorrências (Regra 1)
def plotarGraficoOcorrenciasRegra01(x,y,regra_1):
    x=regra_1['dataMedicao']
    y=regra_1['umidadeRelativa']
    z=df_consulta_bd['dataMedicao']
    w=df_consulta_bd['umidadeRelativa']

    plt.figure(figsize=(15,15))
    plt.scatter(z,w,label = 'Janela', color = 'red', marker = 'o', s = 50)
    plt.scatter(x,y,label = 'Favorabilidade', color = 'black', marker = '.', s = 25)
    plt.style.use('default')
    plt.grid(True)
    plt.title("Gráfico de Favorabildade vs Regra 1", fontsize=15)
    plt.xlabel("Janela de Favorabilidade Selecionada", fontsize=10)
    plt.ylabel("Umidade Relativa", fontsize=10)
    plt.xticks(z,rotation='vertical')
    plt.legend(fontsize=8)
    plt.rcParams['xtick.labelsize'] = 5
    plt.show()
    return z

#Chamada de Função para Plotar gráfico de ocorrências (Regra 1)
if not medianaRegra_01==0:
    z=plotarGraficoOcorrenciasRegra01(x,y,regra_1)
else:
    print("Não há dados para serem plotados para a Regra 1!!")

In [ ]:
# Função para Plotar gráfico de ocorrências (Regra 2)
def plotarGraficoOcorrenciasRegra02(x,y,regra_2):
    x=regra_2['dataMedicao']
    y=regra_2['precipitacao']
    z=df_consulta_bd['dataMedicao']
    w=df_consulta_bd['precipitacao']

    plt.figure(figsize=(15,15))
    plt.scatter(z,w,label = 'Janela', color = 'black', marker = 'o', s = 50)
    plt.scatter(x,y,label = 'Favorabilidade', color = 'red', marker = '.', s = 25)
    plt.style.use('default')
    plt.grid(True)
    plt.title("Gráfico de Favorabildade vs Regra 2", fontsize=15)
    plt.xlabel("Janela de Favorabilidade Selecionada", fontsize=10)
    plt.ylabel("Precipitação", fontsize=10)
    plt.xticks(z,rotation='vertical')
    plt.legend(fontsize=8)
    plt.rcParams['xtick.labelsize']=5
    plt.show()

# Chamada de Função para Plotar gráfico de ocorrências (Regra 2)
if not medianaRegra_02==0:
    plotarGraficoOcorrenciasRegra02(x,y,regra_2)
else:
    print("Não há dados para serem plotados para a Regra 2!!")

In [ ]:
# Função para Plotar gráfico de ocorrências (Regra 3)
def plotarGraficoOcorrenciasRegra03(x,y,regra_3):
    x=regra_3['dataMedicao']
    y=regra_3['temperaturaMaxima']
    z=regra_3['temperaturaMinima']
    a=df_consulta_bd['dataMedicao']
    b=df_consulta_bd['temperaturaMaxima']
    c=df_consulta_bd['temperaturaMinima']
    plt.figure(figsize=(15,15))
    plt.grid(True)

    # Temperatura Máxima
    plt.scatter(a,b,label = 'Janela', color = 'purple', marker = 'o', s = 50)
    plt.scatter(x,y,label = 'Favorabilidade', color = 'red', marker = '.', s = 25)
    plt.title("Gráfico de Favorabilidade vs Regra 3", fontsize=15)
    plt.xlabel("Janela de Favorabilidade Selecionada", fontsize=10)
    plt.ylabel("Temperatura Máxima", fontsize=10)
    plt.xticks(a,rotation='vertical')
    plt.legend(fontsize=8)
    plt.rcParams['xtick.labelsize'] = 5
    plt.show()

    # Temperatura Mínima
    plt.figure(figsize=(15,15))
    plt.scatter(a,c,label = 'Janela', color = 'purple', marker = 'o', s = 50)
    plt.scatter(x,z,label = 'Favorabilidade', color = 'red', marker = '.', s = 25)
    plt.title("Gráfico de Favorabildade vs Regra 3", fontsize=15)
    plt.xlabel("Janela de Favorabilidade Selecionada", fontsize=10)
    plt.ylabel("Temperatura Mínima", fontsize=10)
    plt.xticks(a,rotation='vertical')
    plt.legend(fontsize=8)
    plt.rcParams['xtick.labelsize'] = 5
    plt.grid(True)
    plt.show()

# Chamada de Função para Plotar gráfico de ocorrências (Regra 3)
if type(regra_3)==int:
    if (regra_3==0):
        print("Não há dados para serem plotados para a Regra 3!!")

elif regra_3.empty:
    print("Não há dados para serem plotados para a Regra 3!!")

else:
     plotarGraficoOcorrenciasRegra03(x,y,regra_3)

In [ ]:
# Função para Plotar gráfico de ocorrências (Regra 5)
def plotarGraficoOcorrenciasRegra05(x,y,regra_5):
    x=regra_6['dataMedicao']
    y=regra_6['temperaturaMinima']
    z=df_consulta_bd['dataMedicao']
    w=df_consulta_bd['temperaturaMinima']

    plt.figure(figsize=(15,15))
    plt.grid(True)
    plt.scatter(z,w,label = 'Janela', color = 'green', marker = 'o', s = 50)
    plt.scatter(x,y,label = 'Favorabilidade', color = 'red', marker = '.', s = 25)
    plt.style.use('default')
    plt.title("Gráfico de Favorabildade vs Regra 5", fontsize=15)
    plt.xlabel("Janela de Favorabilidade Selecionada", fontsize=10)
    plt.ylabel("Temperatura Mínima", fontsize=10)
    plt.xticks(z,rotation='vertical')
    plt.legend(fontsize=8)
    plt.rcParams['xtick.labelsize'] = 5
    plt.show()

# Chamada de Função para Plotar gráfico de ocorrências (Regra 5)
#plotarGraficoOcorrenciasRegra05(x,y,regra_5)

In [123]:
# Função para Calcular os erros
    # Abordagens: Figura Mérito e Lógica Fuzzy
def calcularErro(resultado):   
   vetor_resultado_erro=[]
   descricao=["Calculado:","+5% Erro:","-5% Erro:"]

   # Valor Calculado
   vetor_resultado_erro.append(round(resultado, casas_decimais))
   # Considerar +5% de erro
   vetor_resultado_erro.append(round(resultado + ((5*resultado)/100),casas_decimais))
   # Considerar -5% de erro
   vetor_resultado_erro.append(round(resultado - ((5*resultado)/100),casas_decimais))
   
   # Associa o resultado ao erro
   lista_resultado=list(zip(descricao,vetor_resultado_erro))

   return lista_resultado

### Implementação da Figura de Mérito

In [124]:
# Definição do ângulo do triângulo em radianos
# Quantidade de variáveis = "7" para divisão do círculo trigonométrico
# Divisão do Círculo Trigonométrico pelo número de variáveis
angulo = (360/7)

In [125]:
# Círculo de Raio Unitário

def desenharCirculo():
    #-------------------------------------------------
    # Círculo de Raio = 1 (Uso da bibliotecca Matplotlib como opção)
    #c1 = plt.Circle((0, 0), radius=1, edgecolor = 'blue', facecolor = 'none')
    #axes.set_aspect(1) # Usar a mesma escala de dados - plotar unidades para x e y
    #axes.add_patch(c1)
    #-------------------------------------------------

    # (Implementação do Cálculo do Círculo)
    #-------------------------------------------------
    # Círculo de Raio = 1 (Uso da Equação do Cículo)
    # => (x-h)^2 + (y-k)^2 = r^2) => x**2  + y**2 = r**2
    #-------------------------------------------------   
    r = 1
    x = np.linspace(-r,r,150)
    y = np.sqrt(r**2 - x**2)
    plt.plot(x, y,'black')
    plt.plot(x,-y,'black')
    plt.gca().set_aspect('equal')

In [126]:
# Definição dos ângulos para plotar a Figura de Mérito

def definirAngulos(angulo):
        
    # Ângulo inicial = 0
    angulo_0=math.radians(0)
    # Ângulo Eixo 1 = 308.5714 (aproximado)
    angulo_1 = math.radians(360-angulo)
    # Ângulo Eixo 2 = 257.1425 (aproximado)
    angulo_2 = math.radians(360-(angulo*2))
    # Ângulo Eixo 3 = 205.7142 (aproximado)
    angulo_3 = math.radians(360-(angulo*3))
    # Ângulo Eixo 4 = 154.2557 (aproximado)
    angulo_4 = math.radians(360-(angulo*4))
    # Ângulo Eixo 5 = 102.8571 (aproximado)
    angulo_5 = math.radians(360-(angulo*5))
    # Ângulo Eixo 6 = 51.4285 (aproximado)    
    angulo_6 = math.radians(360-(angulo*6))

    return angulo_0, angulo_1, angulo_2, angulo_3, angulo_4, angulo_5, angulo_6

In [127]:
def desenhar_linhas(angulo_0,angulo_1,angulo_2,angulo_3,angulo_4,angulo_5,angulo_6):
    # Definição dos pontos x e y para construir as retas
    #   de acordo com os ângulos de inclinação
    # Ângulo Eixo 0 = 0
    x0 = math.cos(angulo_0) 
    y0 = math.sin(angulo_0)
    # Ângulo Eixo 1 = 308.5714 (aproximado)
    x1 = math.cos(angulo_1) 
    y1 = math.sin(angulo_1)
    # Ângulo Eixo 2 = 257.1425 (aproximado)
    x2 = math.cos(angulo_2)
    y2 = math.sin(angulo_2)
    # Ângulo Eixo 3 = 205.7142 (aproximado)
    x3 = math.cos(angulo_3)
    y3 = math.sin(angulo_3)
    # Ângulo Eixo 4 = 154.2557 (aproximado)
    x4 = math.cos(angulo_4)
    y4 = math.sin(angulo_4)
    # Ângulo Eixo 5 = 102.8571 (aproximado)
    x5 = math.cos(angulo_5)
    y5 = math.sin(angulo_5)
    # Ângulo Eixo 6 = 51.4285 (aproximado)
    x6 = math.cos(angulo_6)
    y6 = math.sin(angulo_6)
    
    return x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6

In [128]:
def desenharEixos(x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6):
    axes = plt.subplot()
    axes.grid(False)

    # Desenhar os eixos

    # Teste: Desenho dos quatro quadrantes no Cículo Unitário
    #axes.plot([0, 1], [0, 0], color = 'black') # (0,1.0), (0,0.0)
    #axes.plot([0,-1], [0, 0], color = 'black') # (0,-1.0), (0,0.0)
    #axes.plot([0, 0], [0, 1], color = 'black') # (0,0.0), (0,1.0)
    #axes.plot([0, 0], [0,-1], color = 'black') # (0,0.0), (0,-1.0)
    
    # Eixo 0
    axes.plot([0, x0], [0, y0], color = 'blue', label= 'Angulo_0')
    #print("Eixo 0: "+ "(0,"+str(x0)+"), (0,"+str(y0)+")")
    # Eixo 1
    axes.plot([0, x1], [0, y1], color = 'purple', label= 'Angulo_1')
    #print("Eixo 1: "+ "(0,"+str(x1)+"), (0,"+str(y1)+")")
    # Eixo 2
    axes.plot([0, x2], [0, y2], color = 'cyan', label = 'Angulo_2')
    #print("Eixo 2: "+"(0,"+str(x2)+"), (0,"+str(y2)+")")
    # Eixo 3
    axes.plot([0, x3], [0, y3], color = 'green', label = 'Angulo_3')
    #print("Eixo 3: "+"(0,"+str(x3)+"), (0,"+str(y3)+")")
    # Eixo 4
    axes.plot([0, x4], [0, y4], color = 'orange', label = 'Angulo_4')
    #print("Eixo 4: "+"(0,"+str(x4)+"), (0,"+str(y4)+")")
    # Eixo 5
    axes.plot([0, x5], [0, y5], color = 'red', label = 'Angulo_5')
    #print("Eixo 5: "+"(0,"+str(x5)+"), (0,"+str(y5)+")")
    # Eixo 6
    axes.plot([0, x6], [0, y6], color = 'yellow', label = 'Angulo_6')
    #print("Eixo 6: "+"(0,"+str(x6)+"), (0,"+str(y6)+")")
    
    return axes

In [129]:
def gerarObjetoPonto(x, y, cor, flag_regra_1, flag_regra_2, flag_regra_3, flag_regra_4, flag_regra_5, flag_regra_6, flag_regra_7):
    
            #   De "flag_regra_1" até "flag_regra_6" o valor deve ser: 'Alta', 'Média' ou 'Baixa'
            #   -> Quando 'Alta' o valor dessa variável deve ser "100%" de favorabilidade e o valor "1" para o plot da Figura de Mérito
            #   -> Quando 'Média' o valor dessa variável deve ser "66.6%" de favorabilidade e o valor "66.6" para o plot da Figura de Mérito
            #   -> Quando 'Baixa' o valor dessa variável deve ser "33.3%" de favorabilidade e o valor "33.3" para o plot da Figura de Mérito
    
     # Verifica se somente a Regra_1 está ativa
    if  flag_regra_1!="":       
        
        if flag_regra_1=="Alta":
            coord_x_reta=x
            coord_y_reta=y
            valor_aproximado=1
            
        if flag_regra_1=="Média":
            coord_x_reta=(x/3)*2
            coord_y_reta=(y/3)*2
            valor_aproximado=0.666

        if flag_regra_1=="Baixa":
            coord_x_reta=x/3
            coord_y_reta=y/3
            valor_aproximado=0.333
    
    # Verifica se somente a Regra_2 está ativa
    if  flag_regra_2!="":
        
        if flag_regra_2=="Alta":
            coord_x_reta=x
            coord_y_reta=y
            valor_aproximado=1
            
        if flag_regra_2=="Média":
            coord_x_reta=(x/3)*2
            coord_y_reta=(y/3)*2
            valor_aproximado=0.666

        if flag_regra_2=="Baixa":
            coord_x_reta=x/3
            coord_y_reta=y/3
            valor_aproximado=0.333
    
    # Verifica se somente a Regra_3 está ativa
    if  flag_regra_3!="":
            
        if flag_regra_3=="Alta":
            coord_x_reta=x
            coord_y_reta=y
            valor_aproximado=1
            
        if flag_regra_3=="Média":
            coord_x_reta=(x/3)*2
            coord_y_reta=(y/3)*2
            valor_aproximado=0.666

        if flag_regra_3=="Baixa":
            coord_x_reta=x/3
            coord_y_reta=y/3
            valor_aproximado=0.333

    # Verifica se somente a Regra_4 está ativa
    if  flag_regra_4!="":
        
        if flag_regra_4=="Alta":
            coord_x_reta=x
            coord_y_reta=y
            valor_aproximado=1
            
        if flag_regra_4=="Média":
            coord_x_reta=(x/3)*2
            coord_y_reta=(y/3)*2
            valor_aproximado=0.666

        if flag_regra_4=="Baixa":
            coord_x_reta=x/3
            coord_y_reta=y/3
            valor_aproximado=0.333                   
    
    # Verifica se somente a Regra_5 está ativa
    if  flag_regra_5!="":
    
        if flag_regra_5=="Alta":
            coord_x_reta=x
            coord_y_reta=y
            valor_aproximado=1
            
        if flag_regra_5=="Média":
            coord_x_reta=(x/3)*2
            coord_y_reta=(y/3)*2
            valor_aproximado=0.666

        if flag_regra_5=="Baixa":
            coord_x_reta=x/3
            coord_y_reta=y/3
            valor_aproximado=0.333

    # Verifica se somente a Regra_6 está ativa
    #   -> Neste caso da variável "Ponto de Orvalho" não foi identificada a condição de "Favorabilidade Média"
    if  flag_regra_6!="":
    
        if flag_regra_6=="Alta":
            coord_x_reta=x
            coord_y_reta=y
            valor_aproximado=1
            
        if flag_regra_6=="Baixa":
            coord_x_reta=x/3
            coord_y_reta=y/3
            valor_aproximado=0.333

    # Verifica se somente a Imagem está ativa
    if  flag_regra_7!="":        
            
        if flag_regra_7=="Alta":
            coord_x_reta=x
            coord_y_reta=y
            valor_aproximado=1
               
        if flag_regra_7=="Baixa":
            coord_x_reta=x/3
            coord_y_reta=y/3
            valor_aproximado=0.333
        
        # Gera o objeto para marcação do ponto de interesse na reta que representa cada
        #  variável analisada no Círculo de Raio Unitário
    
    objeto=plt.Circle((coord_x_reta, coord_y_reta), radius=0.02, edgecolor=cor)

    return objeto,coord_x_reta,coord_y_reta,valor_aproximado

In [130]:
def marcarPontosTracarLinhas(x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,axes):
    # Para cada variável verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente
    #   para marcação dos pontos para traçar as linhas da Figura de Mérito
       
    ### Variável Período Molhamento Foliar
    # Eixo 0
    # Chamada de Função para verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente   
    c_6, coord_x_reta_0, coord_y_reta_0, valor_aproximado_ponto_0 = gerarObjetoPonto(x0,y0,'blue',flag_regra_1,"","","","","","")
    axes.add_artist(c_6)

    #------------------------------------------------
    ## Variável Período Mínimo de Molhamento Foliar
    # Eixo 1
    # Chamada de Função para verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente
    c_7, coord_x_reta_1, coord_y_reta_1, valor_aproximado_ponto_1 = gerarObjetoPonto(x1,y1,'purple',"",flag_regra_2,"","","","","")
    axes.add_artist(c_7)

    #------------------------------------------------
    # Variável Faixa de Temperatura
    # Eixo 2
    # Chamada de Função para verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente
    c_8, coord_x_reta_2, coord_y_reta_2, valor_aproximado_ponto_2 = gerarObjetoPonto(x2,y2,'cyan',"","",flag_regra_3,"","","","")
    # Adicionar o ponto obtido no gráfico
    axes.add_artist(c_8)

    #------------------------------------------------
    # Variável Temperatura Máxima
    # Eixo 3
    # Chamada de Função para verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente
    c_9, coord_x_reta_3, coord_y_reta_3, valor_aproximado_ponto_3 = gerarObjetoPonto(x3,y3,'green',"","","",flag_regra_4,"","","")
    # Adicionar o ponto obtido no gráfico
    axes.add_artist(c_9)

    # ------------------------------------------------
    # Variável Temperatura Mínima
    # Eixo 4
    # Chamada de Função para verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente
    c_10, coord_x_reta_4, coord_y_reta_4, valor_aproximado_ponto_4 = gerarObjetoPonto(x4,y4,'orange',"","","","",flag_regra_5,"","")
    # Adicionar o ponto obtido no gráfico
    axes.add_artist(c_10)

    # ------------------------------------------------
    # Variável Ponto de Orvalho
    # Eixo 5
    # Chamada de Função para verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente
    c_11, coord_x_reta_5, coord_y_reta_5, valor_aproximado_ponto_5 = gerarObjetoPonto(x5,y5,'red',"","","","","",flag_regra_6,"")
    # Adicionar o ponto obtido no gráfico
    axes.add_artist(c_11)

    # ------------------------------------------------
    # Variável Dados do Cultivar/Dados do Classificador da Imagem
    # Eixo 6
    # Chamada de Função para verificar a Porcentagem da variável e retornar o objeto do Ponto correspondente
    c_12, coord_x_reta_6, coord_y_reta_6, valor_aproximado_ponto_6 = gerarObjetoPonto(x6,y6,'yellow',"","","","","","",flag_regra_7)
    # Adicionar o ponto obtido no gráfico
    axes.add_artist(c_12)

    return c_6, c_7, c_8, c_9, c_10, c_11, c_12, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1, coord_y_reta_1,        \
                coord_x_reta_2,coord_y_reta_2, coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4,          \
                coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6, valor_aproximado_ponto_0,              \
                valor_aproximado_ponto_1, valor_aproximado_ponto_2, valor_aproximado_ponto_3, valor_aproximado_ponto_4,\
                valor_aproximado_ponto_5, valor_aproximado_ponto_6

In [131]:
def desenharLinhasFormarFigura(axes, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1, coord_y_reta_1, coord_x_reta_2,coord_y_reta_2,\
        coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4, coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6):
    
    ### Desenhar as linhas entre os pontos para formar a Figura de Mérito
    # Configuração Início->(x1, x0), Fim->(y1, y0) => Variável Período Molhamento Foliar
    axes.plot([(coord_x_reta_1), (coord_x_reta_0)], [coord_y_reta_1, -coord_y_reta_0], color = 'black')
    
    # Configuração Início->(x6, x5), Fim->(y6, y5) => Variável Período Mínimo de Molhamento Foliar
    axes.plot([(coord_x_reta_6), (coord_x_reta_5)], [coord_y_reta_6, coord_y_reta_5], color = 'black')

    # Configuração Início->(x5, x4), Fim->(y5, y4) => Variável Faixa de Temperatura
    axes.plot([(coord_x_reta_5), (coord_x_reta_4)], [coord_y_reta_5, coord_y_reta_4], color = 'black')

    # Configuração Início->(x4, x3), Fim->(y4, y3) => Variável Temperatura Máxima
    axes.plot([(coord_x_reta_4), (coord_x_reta_3)], [coord_y_reta_4, coord_y_reta_3], color = 'black')

    # Configuração Início->(x3, x2), Fim->(y3, y2) => Variável Temperatura Mínima
    axes.plot([(coord_x_reta_3), (coord_x_reta_2)], [coord_y_reta_3, coord_y_reta_2], color = 'black')

    # Configuração Início->(x2, x1), Fim->(y2, y1) => Variável Ponto de Orvalho
    axes.plot([(coord_x_reta_2),(coord_x_reta_1)], [coord_y_reta_2, coord_y_reta_1], color = 'black')

    # Configuração Início->(x0, x6), Fim->(y0, y6) -> Fechamento da Figura (Variável Dados do Cultivar/Dados do Classificador da Imagem)
    axes.plot([(coord_x_reta_0),(coord_x_reta_6)], [coord_y_reta_0, coord_y_reta_6], color = 'black')

In [132]:
# Função para montar identificação das variáveis no Gráfico
def montarIdentifGrafico(x0,x1,x2,x3,x4,x5,x6,y0,y1,y2,y3,y4,y5,y6,axes, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1,\
    coord_y_reta_1, coord_x_reta_2, coord_y_reta_2, coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4,     \
    coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6):
    
    index=0
    x=[x0,x1,x2,x3,x4,x5,x6]
    y=[y0,y1,y2,y3,y4,y5,y6]
    dist_x=[1.02, 1.05, 3.20, 1.66, 1.66, 2.70, 1.05]
    dist_y=[1.0, 1.05, 1.07, 1.20, 1.07, 1.05, 1.02]
    coluna=['Período Molhamento Foliar','Período Mínimo de Molhamento Foliar','Faixa Temperatura',\
        'Temperatura Máxima','Temperatura Mínima','Ponto Orvalho','Dados Cultivar/Imagem']

    # Escreve e organiza as posições das informações das variáveis no gráfico
    for x_1, y_1 in zip(x,y):
        str_xy= '{}'.format(coluna[index], x_1, y_1)
        axes.annotate(str_xy, xy=(x_1*dist_x[index], y_1*dist_y[index]), fontsize=6)
        index+=1

    # Chamada de Função para Desenhar as linhas entre os pontos para formar a Figura de Mérito
    desenharLinhasFormarFigura(axes, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1, coord_y_reta_1, coord_x_reta_2,coord_y_reta_2,  \
        coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4, coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6)
        
    # Estabelecer os limites para exibição da Figura de Mérito no Gráfico
    plt.xlim(-1.5, 1.9)
    plt.ylim(-1.2, 1.2)

    # Plotar o gráfico com a Figura de Mérito
    plt.figsize=(25,25)
    plt.title("Círculo de Raio Unitário - Figura de Mérito")

    # Salvar Figura de Mérito em arquivo
    plt.savefig(path_3 + 'Figura_Merito.png', format='png')
    plt.show()

In [133]:
def plotUnitCircle(angulo_0,angulo_1,angulo_2,angulo_3,angulo_4,angulo_5,angulo_6):
    #Configuração da Plotagem
    plt.style.use("default")
    
    # Chamada de Função para desenhar o Círculo Unitário
    desenharCirculo()
    
    # Chamada de função para desenhar as linhas
    x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6 = desenhar_linhas(angulo_0,\
        angulo_1,angulo_2,angulo_3,angulo_4,angulo_5,angulo_6)
    
    # Chamada de função para desenhar os eixos
    axes=desenharEixos(x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6)
           
    ### Desenhar a Figura de Mérito ###
    # Chamada de função para marcar os pontos para traçar as linhas
    c_6, c_7, c_8, c_9, c_10, c_11, c_12, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1, coord_y_reta_1,   \
    coord_x_reta_2,coord_y_reta_2, coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4,          \
    coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6, valor_aproximado_ponto_0,              \
    valor_aproximado_ponto_1, valor_aproximado_ponto_2, valor_aproximado_ponto_3, valor_aproximado_ponto_4,\
    valor_aproximado_ponto_5, valor_aproximado_ponto_6 = marcarPontosTracarLinhas(x0,y0,x1,y1,x2,y2,x3,y3, \
        x4,y4,x5,y5,x6,y6,axes)

    # Chamada de função para montar identificação das variáveis no Gráfico
    montarIdentifGrafico(x0,x1,x2,x3,x4,x5,x6,y0,y1,y2,y3,y4,y5,y6,axes, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1,\
    coord_y_reta_1, coord_x_reta_2, coord_y_reta_2, coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4,     \
    coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6)

    return c_6, c_7, c_8, c_9, c_10, c_11, c_12, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1, coord_y_reta_1, \
    coord_x_reta_2,coord_y_reta_2, coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4,               \
    coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6, valor_aproximado_ponto_0,                   \
    valor_aproximado_ponto_1, valor_aproximado_ponto_2, valor_aproximado_ponto_3, valor_aproximado_ponto_4,     \
    valor_aproximado_ponto_5, valor_aproximado_ponto_6

In [134]:
def calcular_area_total(valor_aproximado_ponto_0, valor_aproximado_ponto_1, valor_aproximado_ponto_2,     \
    valor_aproximado_ponto_3, valor_aproximado_ponto_4, valor_aproximado_ponto_5, valor_aproximado_ponto_6, 
    angulo, text):
    # Cálculo da área Normalizado - Aproximado
    #  O cálculo considera o valor aproximado do ponto traçado no Gráfico:
    #     => Se o valor normalizado for menor ou igual a 0.33 (considera-se '0.33')
    #     => Se o valor normalizado for menor ou igual a 0.66 (considera-se '0.66')
    #     => Se o valor normalizado for maior que 0.66 (considera-se '1')
    # Entrada nas Funções de Seno --> Ângulo em Radianos

    area_1_n=round((valor_aproximado_ponto_0*valor_aproximado_ponto_1*math.sin(math.radians(angulo)))/2,casas_decimais)
    text+="Resumo dos Cálculos de Área\n----------------------------------------\nArea Triângulo 1-(Variável Período Molhamento Foliar): " \
        + str(area_1_n)
    area_2_n=round((valor_aproximado_ponto_1*valor_aproximado_ponto_2*math.sin(math.radians(angulo)))/2,casas_decimais)
    text+="\nArea Triângulo 2-(Variável Período Mínimo de Molhamento Foliar): "+ str(area_2_n)
    area_3_n=round((valor_aproximado_ponto_2*valor_aproximado_ponto_3*math.sin(math.radians(angulo)))/2,casas_decimais)
    text+="\nArea Triângulo 3-(Variável Faixa de Temperatura): "+ str(area_3_n)
    area_4_n=round((valor_aproximado_ponto_3*valor_aproximado_ponto_4*math.sin(math.radians(angulo)))/2,casas_decimais)
    text+="\nArea Triângulo 4-(Variável Temperatura Máxima): "+ str(area_4_n)
    area_5_n=round((valor_aproximado_ponto_4*valor_aproximado_ponto_5*math.sin(math.radians(angulo)))/2,casas_decimais)
    text+="\nArea Triângulo 5-(Variável Temperatura Mínima): "+ str(area_5_n)
    area_6_n=round((valor_aproximado_ponto_5*valor_aproximado_ponto_6*math.sin(math.radians(angulo)))/2,casas_decimais)
    text+="\nArea Triângulo 6-(Variável Ponto de Orvalho): "+ str(area_6_n)
    area_7_n=round((valor_aproximado_ponto_6*valor_aproximado_ponto_0*math.sin(math.radians(angulo)))/2,casas_decimais)
    text+="\nArea Triângulo 7-(Variável Dados do Cultivar/Dados do Classificador da Imagem): "+ str(area_7_n) + "\n----------------------------------------\n"
    text+="Para Conferir:"
    text+="\n----------------------------------------"
    text+="\n(Triângulo 1): Valores pontos 0 e 1:"+str(valor_aproximado_ponto_0) + "|" + str(valor_aproximado_ponto_1)
    text+="\n(Triângulo 2): Valores pontos 1 e 2:"+str(valor_aproximado_ponto_1) + "|" + str(valor_aproximado_ponto_2)
    text+="\n(Triângulo 3): Valores pontos 2 e 3:"+str(valor_aproximado_ponto_2) + "|" + str(valor_aproximado_ponto_3)
    text+="\n(Triângulo 4): Valores pontos 3 e 4:"+str(valor_aproximado_ponto_3) + "|" + str(valor_aproximado_ponto_4)
    text+="\n(Triângulo 5): Valores pontos 4 e 5:"+str(valor_aproximado_ponto_4) + "|" + str(valor_aproximado_ponto_5)
    text+="\n(Triângulo 6): Valores pontos 5 e 6:"+str(valor_aproximado_ponto_5) + "|" + str(valor_aproximado_ponto_6)
    text+="\n(Triângulo 7): Valores pontos 6 e 0:"+str(valor_aproximado_ponto_6) + "|" + str(valor_aproximado_ponto_0)
    text+="\n----------------------------------------"

    # Cálculo de área total Normalizado - Aproximado
    areaTotal_aprox= round(area_1_n + area_2_n + area_3_n + area_4_n + area_5_n + area_6_n + area_7_n, casas_decimais)
    text+="\nÁrea Total - Figura de Mérito [uA] -> " + str(areaTotal_aprox)

    # Cálculo de área total do Círculo Normalizado - Aproximado
    # -> Considerando que todas as variáveis estão com valor = '1'
    area_total_circulo=round((math.sin(math.radians(angulo))/2)*7, casas_decimais)

    return areaTotal_aprox, area_total_circulo, text

In [135]:
# Função Resultados "Abordagem de Figura de Mérito"
def resultadosFiguraMerito(valor_aproximado_ponto_0, valor_aproximado_ponto_1, valor_aproximado_ponto_2,   \
    valor_aproximado_ponto_3, valor_aproximado_ponto_4, valor_aproximado_ponto_5, valor_aproximado_ponto_6,\
    angulo, text):
    
    # Vetor para armazenar o resultado da Favorabilidade (Figura de Mérito)
    lista_resultado_figura=[]
    
    # Chamada de Função para Calcular a área total (Resultado da Área da Figura de Mérito)
    areaTotal_aprox, area_total_circulo, text = calcular_area_total(valor_aproximado_ponto_0, valor_aproximado_ponto_1, valor_aproximado_ponto_2,\
        valor_aproximado_ponto_3, valor_aproximado_ponto_4, valor_aproximado_ponto_5, valor_aproximado_ponto_6, angulo, text)

    print("Área Total Aproximada:",areaTotal_aprox)

    # Chamada de Função para Calcular o Erro
    lista_resultado=calcularErro(areaTotal_aprox)
       # A variável "lista_resultado" armazena os resultados dos cálculos de área da Função "calcularErro" retornando o valor do "Cálculo de Área"
       #    nesta ordem[0,1,2]: "Calculado:","+5% Erro:","-5% Erro:" 
    
    print("Lista Resultados:",lista_resultado)  
    print("Lista Resultados (5%):",lista_resultado[1])      

    text+="\n\nResultados:\n----------------"
    text+="\nParâmetros.Ref.:(33.3%): " + str(round((area_total_circulo*33.3)/100,casas_decimais)) + " (66.6%): " + str(round((area_total_circulo*66.6)/100,casas_decimais)) + \
                " (Área Total do Círculo): " + str(area_total_circulo)+"\n"

    #Verificar, imprimir e armazenar o "Resultado da Favorabilidade"
        # no index = "1" considerando o resultado "Calculado" SOMENTE com erro de "+5%"

    for index in lista_resultado:
        ##Verifica se o resultado om erro de "+5%" é menor ou igual a 1/3 da área total do círculo (33.3%)
        if index[1] <= (round((area_total_circulo*33.3)/100,casas_decimais)):
            text+="\n\/\/ Favorabilidade Baixa \/\/ - " +str(index)+ " Intervalo (0 até " + str(round(area_total_circulo/3,casas_decimais)) +")"
            lista_resultado_figura.append([index,'Favorabilidade Baixa'])                       

        ##Verifica se o resultado com erro de "+5%" é maior que 1/3 da área total do círculo (0,33%) e menor e igual a 2/3 do total da área (66.6%)
        elif index[1] > round((area_total_circulo*33.3)/100,casas_decimais) and areaTotal_aprox <= round((area_total_circulo*66.6)/100,casas_decimais):
            text+="\n-- Favorabilidade Média -- " +str(index)+ " Intervalo ("+ str(round((area_total_circulo*33.3)/100,casas_decimais)) +" até " + \
                str(round((area_total_circulo*66.6)/100,casas_decimais)) +")"
            lista_resultado_figura.append([index,'Favorabilidade Média'])            

        ##Verifica se o resultado com erro de "+5%" é maior que 2/3 da área total do círculo (66.6%)
        elif index[1] > round((area_total_circulo*66.6)/100,casas_decimais):
            text+="\n^^ Favorabilidade Alta ^^ " +str(index)+ " Intervalo ("+ str(round((area_total_circulo*66.6)/100,casas_decimais)) +" até "+ \
                str(round(area_total_circulo,casas_decimais)) +")"
            lista_resultado_figura.append([index,'Favorabilidade Alta'])            

    print(lista_resultado_figura)
            
    return lista_resultado_figura, lista_resultado, text

In [136]:
# Função para Exibir os valores de entrada da Figura de Mérito
def exibirEntradaFiguraMerito(text):
    text=""
    text+="Dados de Entrada da Figura de Mérito:\n----------------------------------------\n"                               +\
        "Período de Molhamento Foliar (valor):"+str(dados_regras[0])+" (Flag):"+flag_regra_1+"\n"                           +\
        "Período Mínimo de Molhamento Foliar (valor):" + str(dados_regras[1]) +" (Flag):"+flag_regra_2+"\n"                 +\
        "Faixa Temperatura (Mínima):"+str(dados_regras[2])+" e (Máxima):"+str(dados_regras[3])+" (Flag):"+flag_regra_3 +"\n"+\
        "Temperatura Máxima (valor):"+str(dados_regras[4])+" (Flag):"+flag_regra_4+ "\n"                                    +\
        "Temperatura Mínima (valor):"+str(dados_regras[5])+" (Flag):"+flag_regra_5+ "\n"                                    +\
        "Ponto de Orvalho (valor):"+str(dados_regras[6])+" (Flag):"+flag_regra_6+"\n"                                       +\
        "Dados do Cultivar/Imagem (valor):"+str(dados_regras[7])+" (Flag):"+flag_regra_7+"\n"                               +\
        "\n----------------------------------------\n"
    return text

In [ ]:
### Chamadas de Função Abordagem: Figura de Mérito ###

# Chamada de função para definição dos ângulos para o processo
angulo_0,angulo_1,angulo_2,angulo_3,angulo_4,angulo_5,angulo_6 = definirAngulos(angulo)

# Chamada de função para desenhar a Figura de Mérito
c_6, c_7, c_8, c_9, c_10, c_11, c_12, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1, coord_y_reta_1,        \
    coord_x_reta_2,coord_y_reta_2, coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4,           \
    coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6, valor_aproximado_ponto_0,               \
    valor_aproximado_ponto_1, valor_aproximado_ponto_2, valor_aproximado_ponto_3, valor_aproximado_ponto_4, \
    valor_aproximado_ponto_5, valor_aproximado_ponto_6 = plotUnitCircle(angulo_0,angulo_1,angulo_2,angulo_3,\
    angulo_4,angulo_5,angulo_6)

# Chamada de Função para Exibir os valores de entrada da Figura de Mérito
text=exibirEntradaFiguraMerito(text)

# Chamada de função para calcular os resultados da Figura de Mérito
lista_resultado_figura, lista_resultado, text=resultadosFiguraMerito(valor_aproximado_ponto_0, valor_aproximado_ponto_1, valor_aproximado_ponto_2,\
    valor_aproximado_ponto_3, valor_aproximado_ponto_4, valor_aproximado_ponto_5, valor_aproximado_ponto_6, angulo, text)

In [138]:
#Implementação da Lógica Fuzzy
#Descrição do Modelo Fuzzy:

#Entradas (antecedentes):

## Variável (1) - Período Molhamento Foliar =>Se período de Molhamento Foliar (umidade relativa) maior ou igual a 90%
#Universo (intervalo de valores crisp): 0 a 100 (%)
#Conjunto difuso (valores difusos): umidade baixa, umidade média, umidade favorabilidade
#---------------------------------------------------------
## Variável (2) - Período Mínimo de Molhamento Foliar => Se período de Molhamento Foliar maior que 6h - equivalente a 1/4 de 100%
    # (Precipitação maior ou igual a 25%)
#Universo (intervalo de valores crisp): 0 a 100 (%)
#Conjunto difuso (valores difusos): pouco tempo, tempo favorabilidade, muito tempo
#---------------------------------------------------------
## Variável (3) - Dados Cultivar/Imagem
#Universo (intervalo de valores crisp): 0 ou 1
#Conjunto difuso (valores difusos): com favorabilidade, sem favorabilidade
#---------------------------------------------------------
## Variável (4) - Ponto Orvalho => Temperatura Média Condensada < 2 (Graus Celsius) que o Ponto de Orvalho
#Universo (intervalo de valores crisp): -1 a 3 (Graus Celsius)
#Conjunto difuso (valores difusos): temperatura baixa, temperatura favorabilidade, temperatura alta
#---------------------------------------------------------
## Variável (5) - Faixa Temperatura (dividida em Temperatura Inicial e Temperatura Final)
#Universo (intervalo de valores crisp): Inicial:0 a 27 (Graus Celsius) e Final:13 a 44 (Graus Celsius)
#Conjunto difuso (valores difusos): temperatura baixa, temperatura favorabilidade, temperatura alta
#---------------------------------------------------------
## Variável (6) - Temperatura Mínima
#Universo (intervalo de valores nítidos/crisp): 0 a 27 (Graus Celsius)
#Conjunto difuso (valores difusos): temperatura mínima baixa, temperatura minima favorabilidade, temperatura minima alta
#---------------------------------------------------------
## Variável (7) - Temperatura Máxima
#Universo (intervalo de valores crisp): 13 a 44 (Graus Celsius)
#Conjunto difuso (valores difusos): temperatura maxima baixa, temperatura maxima favorabilidade, temperatura maxima alta
#---------------------------------------------------------
#Saída (consequentes): Risco
#Universo (valores crisp): 0 a 100%
#Conjunto difuso (valores difusos): baixa, mdia, alta
#---------------------------------------------------------

####Regras de Decisão####

#SE Período de Molhamento Foliar (umidade relativa) maior ou igual a 90% ENTÃO favorabilidade igual TRUE
#SE Período Mínimo de Molhamento Foliar maior que 6h (equivalente a 1/4 de 100% de Umidade Relativa) E \
#   (Precipitação maior ou igual a 25%) ENTÃO favorabilidade igual TRUE
#SE a Faixa de Temperatura estiver entre 18 a 25°C ENTÃO favorabilidade igual TRUE
#SE a Faixa de Temperatura estiver entre 18 a 26.5°C ENTÃO favorabilidade igual TRUE
#SE temperatura máxima ou mínima menor (2 graus) que o ponto de orvalho ENTÃO favorabilidade igual TRUE
#SE imagem tem a doença ENTÃO favorabilidade igual TRUE
# -------------------------------------------------------------------------------------------------------
#SE favorabilidade for TRUE para até duas variáveis (0,33%) ENTÃO favorabilidade baixa
#SE favorabilidade for TRUE para quatro variáveis (0,66%) ENTÃO favorabilidade média
#SE favorabilidade for TRUE para acima de quatro (100%) variáveis ENTÃO favorabilidade alta

In [139]:
## Função para definição de Variáveis do Problema
def definirVariaveisFuzzy():
    # Variável Período Molhamento Foliar
    periodoMolhamentoFoliar=ctrl.Antecedent(np.arange(0, 101, 1), 'periodoMolhamentoFoliar')
    # Variável Período Mínimo Molhamento Foliar
    periodoMinMolhamentoFoliar=ctrl.Antecedent(np.arange(0, 101, 1), 'periodoMinMolhamentoFoliar')
    # Variável Dados Cultivar/Imagem
    dadosCultivarImagem=ctrl.Antecedent(np.arange(0, 2, 1), 'dadosCultivarImagem')
    # Variável Ponto Orvalho
    pontoOrvalho=ctrl.Antecedent(np.arange(-2, 5, 1), 'pontoOrvalho')
    # Variável Faixa Temperatura Inicial
    faixaTemperaturaInicial=ctrl.Antecedent(np.arange(0, 28, 1), 'faixaTemperaturaInicial')
    # Variável Faixa Temperatura Final
    faixaTemperaturaFinal=ctrl.Antecedent(np.arange(14, 45, 1), 'faixaTemperaturaFinal')
    # Variáveis Temperatura Mínima
    temperaturaMinima=ctrl.Antecedent(np.arange(0, 28, 1), 'temperaturaMinima')
    # Variáveis Temperatura Máxima
    temperaturaMaxima=ctrl.Antecedent(np.arange(14, 46, 1), 'temperaturaMaxima')

    # Variável de Saída
    favorabilidade=ctrl.Consequent(np.arange(0, 101, 1), 'favorabilidade')

    return periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade

## Chamada de função para definição de Variáveis do Problema
periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade=definirVariaveisFuzzy()

In [140]:
# Função para criar as funções de pertinência tipo triângulo (mapeamento entre valores nítidos e difusos)
def funçoesPertinencia(periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade):
    #Definições das variáveis (funções de pertinência) pelo Processo Manual

    ### Período Molhamento Foliar =>Se período de Molhamento Foliar (umidade relativa) igual ou maior que 90%
    #Universo (intervalo de valores crisp): 0 a 100 (%)
    #Conjunto difuso (valores difusos): umidade baixa, umidade média, umidade favorabilidade
    periodoMolhamentoFoliar['umidade abaixo do limiar'] = fuzz.trimf(periodoMolhamentoFoliar.universe, [0, 43, 89])
    periodoMolhamentoFoliar['umidade no limiar'] = fuzz.trimf(periodoMolhamentoFoliar.universe, [88, 90, 94])
    periodoMolhamentoFoliar['umidade acima do limiar'] = fuzz.trimf(periodoMolhamentoFoliar.universe, [93, 96, 100])

    ## Variável Período Mínimo de Molhamento Foliar => Se período de Molhamento Foliar maior que 6h - equivalente a 1/4 de 100%
        # (Precipitação igual ou maior que 25%)
    #Universo (intervalo de valores crisp): de 25% em diante (limite 70%)
    #Conjunto difuso (valores difusos): pouco tempo, tempo favorabilidade, muito tempo
    periodoMinMolhamentoFoliar['tempo abaixo do limiar'] = fuzz.trimf(periodoMinMolhamentoFoliar.universe, [0, 14, 24])
    periodoMinMolhamentoFoliar['tempo no limiar'] = fuzz.trimf(periodoMinMolhamentoFoliar.universe, [22, 46, 70])
    periodoMinMolhamentoFoliar['tempo acima do limiar'] = fuzz.trimf(periodoMinMolhamentoFoliar.universe, [66, 83, 100])

    ## Variável Dados Cultivar/Imagem
    #Universo (intervalo de valores crisp): 0 ou 1
    #Conjunto difuso (valores difusos): com favorabilidade, sem favorabilidade
    dadosCultivarImagem['não favorável'] = fuzz.trimf(dadosCultivarImagem.universe, [0, 0, 1])
    dadosCultivarImagem['favorável'] = fuzz.trimf(dadosCultivarImagem.universe, [1, 1, 1])

    ## Variável Ponto Orvalho (intervalo Temperatura Média) - Temp. Média menor (2 Graus) Celsius que o Ponto de Orvalho
    #Universo (intervalo de valores crisp): -1 a 3 (Graus Celsius)
    #Conjunto difuso (valores difusos): temperatura baixa, temperatura favorabilidade, temperatura alta
    pontoOrvalho['temperatura abaixo do limiar'] = fuzz.trimf(pontoOrvalho.universe, [-2, -1, 0])
    pontoOrvalho['temperatura no limiar'] = fuzz.trimf(pontoOrvalho.universe, [0, 1, 2])
    pontoOrvalho['temperatura acima do limiar'] = fuzz.trimf(pontoOrvalho.universe, [2, 3, 4])

    ## Variável Faixa Temperatura (divididos em Temperatura Inicial e Temperatura Final)
    #Universo (intervalo de valores crisp): Inicial:0 a 27 Graus Celsius e Final:13 a 44 Graus Celsius
    #Conjunto difuso (valores difusos): temperatura baixa, temperatura favorabilidade, temperatura alta
    faixaTemperaturaInicial['faixa abaixo do limiar'] = fuzz.trimf(faixaTemperaturaInicial.universe,  [0, 7, 15])
    faixaTemperaturaInicial['faixa no limiar'] = fuzz.trimf(faixaTemperaturaInicial.universe, [14.4, 18, 21.4])
    faixaTemperaturaInicial['faixa acima do limiar'] = fuzz.trimf(faixaTemperaturaInicial.universe, [21, 24, 27])
    faixaTemperaturaFinal['faixa abaixo do limiar'] = fuzz.trimf(faixaTemperaturaFinal.universe, [14, 19, 24])
    faixaTemperaturaFinal['faixa no limiar'] = fuzz.trimf(faixaTemperaturaFinal.universe, [23.4, 26, 28.4])
    faixaTemperaturaFinal['faixa acima do limiar'] = fuzz.trimf(faixaTemperaturaFinal.universe, [28, 36, 44])

    ## Variáveis Temperatura Mínima
    #Universo (intervalo de valores crisp): 1 a 26 Graus Celsius
    #Conjunto difuso (valores difusos): temperatura mínima baixa, temperatura minima favorabilidade, temperatura minima alta
    temperaturaMinima['temperatura minima abaixo do limiar'] = fuzz.trimf(temperaturaMinima.universe, [0, 7, 15])
    temperaturaMinima['temperatura minima no limiar'] = fuzz.trimf(temperaturaMinima.universe, [14, 18, 22])
    temperaturaMinima['temperatura minima acima do limiar'] = fuzz.trimf(temperaturaMinima.universe, [21, 24, 27])

    ## Variáveis Temperatura Máxima
    #Universo (intervalo de valores crisp): de 14 a 44 Graus Celsius
    #Conjunto difuso (valores difusos): temperatura maxima baixa, temperatura maxima favorabilidade, temperatura maxima alta
    temperaturaMaxima['temperatura maxima abaixo do limiar'] = fuzz.trimf(temperaturaMaxima.universe, [14, 19, 24])
    temperaturaMaxima['temperatura maxima no limiar'] = fuzz.trimf(temperaturaMaxima.universe, [23, 26, 28])
    temperaturaMaxima['temperatura maxima acima do limiar'] = fuzz.trimf(temperaturaMaxima.universe, [27, 35, 45])

    #Saída (consequentes): Favorabilidade
    #Universo (valores nítidos/crisp): 0 a 100%
    #Conjunto difuso (valores difusos): baixa, média, alta
    favorabilidade['baixa'] = fuzz.trimf(favorabilidade.universe, [0, 17.15, 33.3])
    favorabilidade['média'] = fuzz.trimf(favorabilidade.universe, [32.3, 50, 67.6])
    favorabilidade['alta'] = fuzz.trimf(favorabilidade.universe, [66.6, 84, 100])

# Chamada de função para criar as funções de pertinência tipo triângulo (mapeamento entre valores nítidos e difusos)
funçoesPertinencia(periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade)

In [ ]:
# Função para Visualização gráfica das Funções de Pertinência
def visualizarFuncoesPertinencia(periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade):
    periodoMinMolhamentoFoliar.view()
    periodoMolhamentoFoliar.view()
    dadosCultivarImagem.view()
    pontoOrvalho.view()
    faixaTemperaturaInicial.view()
    faixaTemperaturaFinal.view()
    temperaturaMinima.view()
    temperaturaMaxima.view()
    favorabilidade.view()

# Chamada de Função para Visualização gráfica das Funções de Pertinência
#visualizarFuncoesPertinencia(periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
#        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade)

In [142]:
# Elaboração das Regras de Decisão Fuzzy

# Conjunto de Regras:
#--------------------

#SE Período de Molhamento Foliar (umidade relativa) maior ou igual a 90% ENTÃO favorabilidade igual TRUE
#SE Período Mínimo de Molhamento Foliar maior que 6h (equivalente a 1/4 de 100% de Umidade Relativa) E
#   (Precipitação maior ou igual a 25%) ENTÃO favorabilidade igual TRUE
#SE a Faixa de Temperatura estiver entre 18 a 25°C ENTÃO favorabilidade igual TRUE
#SE a Faixa de Temperatura estiver entre 18 a 26.5°C ENTÃO favorabilidade igual TRUE
#SE diferença da Temperatura Média for menor (2 graus) que o Ponto de Orvalho ENTÃO favorabilidade igual TRUE 
#SE Maior Temperatura Máxima estiver dentro da faixa de 18 a 26 graus então favorabilidade igual TRUE
#SE Menor Temperatura Mínima estiver dentro da faixa de 18 a 26 graus então favorabilidade igual TRUE
#SE imagem tem a doença ENTÃO favorabilidade igual TRUE

# -------------------------------------------------------------------------------------------------------
# Tabela lógica de Regras:
#-------------------------

#SE favorabilidade for TRUE para até duas variáveis ENTÃO favorabilidade baixa
    #(1 opção: variável 1 ou variável 2 ou variável 3 ou variável 4 ou variável 5 ou variável 6 ou variável 7
    #(2 opções: variável 1 ou grupo(variável 2 ou variável 3 ou variável 4 ou variável 5 ou variável 6 ou variável 7)
        
#SE favorabilidade for TRUE para até quatro variáveis ENTÃO favorabilidade média
    #(3 opções: variável 1 E variável 2 E grupo(variável 3 ou variável 4 ou variável 5 ou variável 6 ou variável 7)
    #(4 opções: variável 1 E variável 2 E variável 3 E grupo(variável 4 ou variável 5 ou variável 6 ou variável 7)

#SE favorabilidade for TRUE para acima de quatro variáveis ENTÃO favorabilidade alta
    #(5 opções: variável 1 E variável 2 E variável 3 E variável 4 E grupo(variável 5 ou variável 6 ou variável 7)
    #(6 opções: variável 1 E variável 2 E variável 3 E variável 4 E variável 5 E grupo(variável 6 ou variável 7)
    #(7 opções: variável 1 E variável 2 E variável 3 E variável 4 E variável 5 E variável 6 E variável 7
# -------------------------------------------------------------------------------------------------------

        # PARÂMETROS (Considerados para a implementação das Regras Fuzzy)
        #    ###(favorabilidade['baixa'])###
        #    periodoMinMolhamentoFoliar['tempo abaixo do limiar'] 
        #    dadosCultivarImagem['não favorável'] 
        #    pontoOrvalho['temperatura abaixo do limiar']
        #    (faixaTemperaturaInicial['faixa abaixo do limiar'] & faixaTemperaturaFinal['faixa abaixo do limiar']) 
        #    TemperaturaMinima['temperatura minima abaixo do limiar'] 
        #    TemperaturaMaxima['temperatura maxima abaixo do limiar']
        #  -----------------------------------------------------------------------------------
        #    ###(favorabilidade['média'])###
        #    periodoMinMolhamentoFoliar['tempo acima do limiar'] 
        #    periodoMolhamentoFoliar['umidade no limiar'] 
        #    dadosCultivarImagem['não favorável']
        #    pontoOrvalho['temperatura acima do limiar']
        #    (faixaTemperaturaInicial['faixa acima do limiar'] & faixaTemperaturaFinal['faixa acima do limiar'])
        #    TemperaturaMinima['temperatura minima acima do limiar']
        #    TemperaturaMaxima['temperatura maxima acima do limiar']
        #  -----------------------------------------------------------------------------------
        #    ###(favorabilidade['alta'])###
        #    periodoMinMolhamentoFoliar['tempo no limiar'] 
        #    periodoMolhamentoFoliar['umidade no limiar'] 
        #    dadosCultivarImagem['favorável']
        #    pontoOrvalho['temperatura no limiar']
        #    (faixaTemperaturaInicial['faixa no limiar'] & faixaTemperaturaFinal['faixa no limiar'])
        #    TemperaturaMinima['temperatura minima no limiar']
        #    TemperaturaMaxima['temperatura maxima no limiar']
        #-----------------------------------------------------

In [143]:
# # Implementação das Regras Fuzzy

# Chamada de Função para definição das Regras Fuzzy 
rule1_1, rule1_2, rule1_3, rule1_4, rule1_5, rule1_6, rule1_7, rule1_8,\
        rule2_1, rule2_2, rule2_3, rule2_4, rule2_5, rule2_6, rule2_7, rule2_8, rule2_9, rule2_10,rule2_11, \
        rule2_12, rule2_13, rule2_14, rule2_15, rule2_16, rule2_17, rule2_18, rule2_19, rule2_20, rule2_21, \
        rule3_1,rule3_2,rule3_3,rule3_4,rule3_5,rule3_6,rule3_7,rule3_8,rule3_9,rule3_10,rule3_11,rule3_12, \
        rule3_13,rule3_14,rule3_15,rule3_16, rule3_17,rule3_18,rule3_19,rule3_20,rule3_21,rule3_22,rule3_23,\
        rule3_24,rule3_25,rule3_26,rule3_27,rule3_28,rule3_29,rule3_30,rule3_31, rule3_32,rule3_33,rule3_34,rule3_35, \
        rule4_1,rule4_2,rule4_3,rule4_4,rule4_5,rule4_6,rule4_7,rule4_8,rule4_9,rule4_10,rule4_11,rule4_12,rule4_13,  \
        rule4_14,rule4_15,rule4_16,rule4_17,rule4_18,rule4_19,rule4_20,rule4_21,rule4_22,rule4_23,rule4_24,rule4_25,  \
        rule4_26,rule4_27,rule4_28,rule4_29,rule4_30,rule4_31,rule4_32,rule4_33,rule4_34,rule4_35,\
        rule5_1, rule5_2, rule5_3, rule5_4, rule5_5,rule5_6, rule5_7, rule5_8, rule5_9, rule5_10,rule5_11, rule5_12,  \
        rule5_13, rule5_14, rule5_15,rule5_16, rule5_17, rule5_18, rule5_20,rule5_21= regras.regras(
        periodoMinMolhamentoFoliar,periodoMolhamentoFoliar,dadosCultivarImagem,pontoOrvalho,faixaTemperaturaInicial,
        faixaTemperaturaFinal,temperaturaMinima,temperaturaMaxima, favorabilidade)

In [144]:
# Função para Simulação do Controlador Fuzzy
def simularControladorFuzzy():
   favorabilidade_ctrl = ctrl.ControlSystem([
      # Regras de Favorabilidade Baixa (1 e 2 opções)
      rule1_1,rule1_2,rule1_3,rule1_4,rule1_5,rule1_6,rule1_7,rule1_8,   
      #---------------------------------------------------------------------
      # Regras de Favorabilidade Média (3 opções)
      rule2_1, rule2_2, rule2_3, rule2_4, rule2_5, rule2_6, rule2_7, rule2_8, rule2_9,
      rule2_10,rule2_11,rule2_12, rule2_13, rule2_14, rule2_15, rule2_16, rule2_17, rule2_18,
      rule2_19, rule2_20, rule2_21,   
      #---------------------------------------------------------------------
      # Regras de Favorabilidade Média (4 opções)
      rule3_1,rule3_2,rule3_3,rule3_4,rule3_5,rule3_6,rule3_7,rule3_8,rule3_9,rule3_10,rule3_11,rule3_12,
      rule3_13,rule3_14,rule3_15,rule3_16,rule3_17,rule3_18,rule3_19,rule3_20,rule3_21,rule3_22,rule3_23,rule3_24,
      rule3_25,rule3_26,rule3_27,rule3_28,rule3_29,rule3_30,rule3_31,rule3_32,rule3_33,rule3_34,rule3_35,
      #---------------------------------------------------------------------
      # Regras de Favorabilidade Alta (5 opções)
      rule4_1,rule4_2,rule4_3,rule4_4,rule4_5,rule4_6,rule4_7,rule4_8,rule4_9,rule4_10,rule4_11,rule4_12,rule4_13,
      rule4_14,rule4_15,rule4_16,rule4_17,rule4_18,rule4_19,rule4_20,rule4_21,rule4_22,rule4_23,rule4_24,rule4_25,rule4_26,
      rule4_27,rule4_28,rule4_29,rule4_30,rule4_31,rule4_32,rule4_33,rule4_34,rule4_35,
      #---------------------------------------------------------------------
      # Regras de Favorabilidade Alta (6 e 7 opções)
      rule5_1,rule5_2,rule5_3,rule5_4,rule5_5,rule5_6,rule5_7,rule5_8,rule5_9,rule5_10,rule5_11,rule5_12,
      rule5_13,rule5_14,rule5_15,rule5_16,rule5_17,rule5_18,rule5_20,rule5_21,
      #---------------------------------------------------------------------
   ])

   return favorabilidade_ctrl

# Chamada de Função para Simulação do Controlador Fuzzy
#favorabilidade_ctrl=simularControladorFuzzy()

In [145]:
# Função para Processar as regras no controlador Fuzzy
def processarRegrasFuzzy(favorabilidade_ctrl):
   favorabilidade_simulador = ctrl.ControlSystemSimulation(favorabilidade_ctrl)

   # Entradas (valor das variáveis) para a simulação do Modelo Fuzzy
   favorabilidade_simulador.input['periodoMolhamentoFoliar'] = medianaRegra_01
   favorabilidade_simulador.input['periodoMinMolhamentoFoliar'] =  medianaRegra_02
   favorabilidade_simulador.input['faixaTemperaturaInicial'] = medianaRegra_03_1
   favorabilidade_simulador.input['faixaTemperaturaFinal'] = medianaRegra_03_2
   favorabilidade_simulador.input['temperaturaMinima'] = regra_5_valor
   favorabilidade_simulador.input['temperaturaMaxima'] = regra_4_valor
   favorabilidade_simulador.input['pontoOrvalho'] = medianaRegra_06
   favorabilidade_simulador.input['dadosCultivarImagem'] = regra_7 

   # Processo Defuzificação
   favorabilidade_simulador.compute()
   resultado=round(favorabilidade_simulador.output['favorabilidade'],casas_decimais)

   return favorabilidade_simulador, resultado

# Chamada de Função para Processar as regras no controlador Fuzzy
#favorabilidade_simulador, resultado=processarRegrasFuzzy(favorabilidade_ctrl)

In [146]:
# Função para Mostrar os Resultados Fuzzy
def mostrarResultadosFuzzy(lista_resultado,favorabilidade,favorabilidade_simulador, text):
    
    text+="\n\nEntradas para simulação do Modelo Fuzzy:\n---------------------------------------\nPeríodo Molhamento Foliar:" + str(medianaRegra_01)+\
      "\nPeríodo Min. Molhamento Foliar:"+str(medianaRegra_02)+"\nFaixa Temperatura Inicial:"+ str(medianaRegra_03_1)+\
      "\nFaixa Temperatura Final:"+str(medianaRegra_03_2)+"\nTemperatura Minima:" +str(regra_5_valor)+\
      "\nTemperatura Maxima:"+str(regra_4_valor)+"\nPonto Orvalho:"+str(medianaRegra_06)+"\nDados Cultivar Imagem:"+str(regra_7)
        
    lista_resultado_fuzzy=[]
    text+="\n\nResultados Abordagem Fuzzy:\n--------------------------------"

    # Considera o index = "1" da "lista_resultado" porque somente calcula para o erro de "+5%"

    for index in lista_resultado:
        if index[1]<=33.3:
            text+="\n"+str(index)+":Previsão Fuzzy => Favorabilidade Baixa\n-----------------------------\n"
            lista_resultado_fuzzy.append([index,'Favorabilidade Baixa'])
            
        elif (index[1]>33.3) & (index[1]<=66.6):
            text+="\n"+str(index)+":Previsão Fuzzy => Favorabilidade Média\n-----------------------------\n"
            lista_resultado_fuzzy.append([index,'Favorabilidade Média'])
            
        elif index[1]>66.6:
            text+="\n"+str(index)+":Previsão Fuzzy => Favorabilidade Alta\n-----------------------------\n"
            lista_resultado_fuzzy.append([index,'Favorabilidade Alta'])
            
    # Resultado Gráfico - Favorabilidade Abordagem Fuzzy
    favorabilidade.view(sim=favorabilidade_simulador)
    # Salvar Figura de Mérito em arquivo
    plt.savefig(path_3 + 'Grafico_Fuzzy.png', format='png')
        
    return lista_resultado_fuzzy, text

In [ ]:
# Chamada de Função para o Cálculo dos Erros:
lista_resultado=calcularErro(resultado)

# Chamada de Função para Mostrar os Resultados Fuzzy
lista_resultado_fuzzy, text=mostrarResultadosFuzzy(lista_resultado,favorabilidade,favorabilidade_simulador,text)

### Modelo da Abordagem Cadeia Oculta de Markov

In [ ]:
# Exibição do Modelo Gráfico
img =cv2.cvtColor(cv2.imread('Automato_3_alta.png'), cv2.COLOR_BGR2RGB)
plt.figure(figsize=[10, 10])
plt.imshow(img)

In [149]:
# Carregar as combinações - Tabela Verdade
    # Período de Molhamento Foliar (V1);
    # Período Mínimo de Molhamento Foliar (V2);
    # Faixa de Temperatura (V3);
    # Temperatura Máxima (V4);
    # Temperatura Mínima (V5);
    # Ponto de Orvalho (V6);
    # Dados de Classificação da Imagem (V7);
    # -------------
    # Saída - Indica qual Estado corresponde a combinação (S).

# Carregar as Probabilidades das Observações:
    # Probabilidade - Período de Molhamento Foliar (P_V1);
    # Probabilidade - Período Mínimo de Molhamento Foliar (P_V2);
    # Probabilidade - Faixa de Temperatura (P_V3);
    # Probabilidade - Temperatura Máxima (P_V4);
    # Probabilidade - Temperatura Mínima (P_V5);
    # Probabilidade - Ponto de Orvalho (P_V6);
    # Probabilidade - Dados de Classificação da Imagem (P_V7).

# Função para Carregar a Tabela Verdade (Abordagem Markov)
def carregarTabelaVerdade():
    # Leitura dos dados da Tabela Verdade - Arquivo .CSV
    # Carga dos dados em um Dataframe Pandas
    path="Tabelas_Verdade_Probabilidades_7_VAR.csv"
    df_tab_verdade=pd.DataFrame()
    df_tab_verdade=pd.read_csv(path, sep=';')
    
    return df_tab_verdade

# Chamada de Função para Carregar a Tabela Verdade (Abordagem Markov)
df_tab_verdade=carregarTabelaVerdade()

In [ ]:
# Função para encontrar a Combinação da Tabela - Descobrir o Estado correspondente
# Dado o vetor de Ocorrências de Favorabilidade da FAS
def encontraCombinacao(ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,ocorrenciasRegra_4,ocorrenciasRegra_5,ocorrenciasRegra_6,
                    ocorrenciasRegra_7, text, coluna_vetor_ocorrencias):
    vetor_compara=[]
    vetor_ocorrencias=[ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,ocorrenciasRegra_4,ocorrenciasRegra_5,ocorrenciasRegra_6,
                    ocorrenciasRegra_7]
    
    text+="\nEntradas para simulação Modelo Cadeias Ocultas de Markov"
    text+="\nVetor de Ocorrências:" + str(vetor_ocorrencias)

    for ocorrencia in vetor_ocorrencias:
        if ocorrencia>0:
            vetor_compara.append(1)
        else:
            vetor_compara.append(0)
    text+="\nVetor de Comparação:" + str(vetor_compara)

    # Recupera na Tabela Verdade os dados iguais ao Vetor de Comparação
    vetor_resultado=df_tab_verdade[(df_tab_verdade['V1']==vetor_compara[0]) & (df_tab_verdade['V2']==vetor_compara[1]) &              \
        (df_tab_verdade['V3']==vetor_compara[2]) & (df_tab_verdade['V4']==vetor_compara[3]) &                                         \
        (df_tab_verdade['V5']==vetor_compara[4]) & (df_tab_verdade['V6']==vetor_compara[5]) & (df_tab_verdade['V7']==vetor_compara[6])]
    display(vetor_resultado)

    # Acumular os valores do Vetor de Ocorrências
    coluna_vetor_ocorrencias.append(vetor_ocorrencias)

    return vetor_resultado, vetor_compara, vetor_ocorrencias, text, coluna_vetor_ocorrencias

# Chamada de Função para encontrar a Combinação da Tabela - Descobrir o Estado correspondente
#vetor_resultado, vetor_compara, vetor_ocorrencias, text, coluna_vetor_ocorrencias=encontraCombinacao(ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,\
#    ocorrenciasRegra_4,ocorrenciasRegra_5,ocorrenciasRegra_6,ocorrenciasRegra_7, text, coluna_vetor_ocorrencias)

In [151]:
# Função para capturar o índice do resultado do "vetor_resultado"
def capturarIndiceFavorab(vetor_resultado, text):
        #  Origem da Tabela Verdade de Favorabilidade (Abordagem Markov)
    ID_TAB_VERD_FAVORAB=0
    ID_TAB_VERD_FAVORAB=vetor_resultado.index.values
    ID_TAB_VERD_FAVORAB[0]+=1
    text+="\n\nResultados Abordagem Cadeias Ocultas de Markov"
    text+="\n-------------------------------------------------"
    text+="\nÍndice Resultado Tabela Verdade:\n" + str(ID_TAB_VERD_FAVORAB[0])

    return text

# Chamada de Função para capturar o índice do resultado do "vetor_resultado"
#text=capturarIndiceFavorab(vetor_resultado,text)

In [152]:
# Função para Normalização dos valores de probabilidades iniciais
def notmalizaProbabilidadesIniciais():
    # Probabilidade baixa (0 - 33,3)
    # Probabilidade média (33,4 - 66,6)
    # Probabilidade alta (66,7 - 100)
    scala_Regras=MinMaxScaler(feature_range=(0,1))
    PBI=scala_Regras.fit_transform(np.array([0, 34.4, 66.7]).reshape(-1, 1))
    print(PBI)

    return PBI

In [153]:
# Função para identificar a probabilidade do Estado Atual
def probabilidadeEstadoAtual(text):
    # Inserir informações do Modelo 
    # Matriz de Probabilidade Inicial
    ##probab_inicial= [[0.1], [0.2], [0.7]]
    probab_inicial= [[0.33], [0.33], [0.33]]

    # Matriz de Transição de Estados
    matriz_transicao= [[0.4, 0.3, 1],
                    [0.6, 0.7, 0],
                    [0  , 0  , 0]]

    # Tratamento das Probabilidades - Combinações das Variáveis
    probab_estado=vetor_resultado[['P_V1','P_V2','P_V3','P_V4','P_V5','P_V6','P_V7']]
    text+=str(probab_estado)

    # Soma dos valores das Probabilidades (Soma = "1")
    probab_previsao=np.sum(probab_estado)

    return probab_estado, probab_previsao, matriz_transicao, probab_inicial, text

# Chamada de Função para identificar a probabilidade do Estado Atual
#probab_estado, probab_previsao, matriz_transicao, probab_inicial, text=probabilidadeEstadoAtual(text)

In [154]:
# Função para identificar o Estado do Atual da Favorabilidade FAS
def identificaEstadoAtual(vetor_resultado, probab_previsao, text):
    # Atribui o valor da Matriz de Previsão
    # -> uso após o primeiro cálculo de previsão
    # -> monta a matriz de previsão com a soma dos valores
    #    das probabilidades origidas da combinação escolhida
    # Imprimir o resultado

    if np.array(vetor_resultado['S'])==1:
        resultado_markov= "Favorabilidade Baixa"
        text+="\nAbordagem Cadeias Ocultas de Markov - Estado Atual:" + str(resultado_markov) + "\n\n"
        previsao_prox=[[probab_previsao.sum()], [0], [0]]
    elif np.array(vetor_resultado['S'])==2:
        resultado_markov= "Favorabilidade Média"
        text+="\nAbordagem Cadeias Ocultas de Markov - Estado Atual:" + str(resultado_markov) + "\n\n"
        previsao_prox=[ [0], [probab_previsao.sum()], [0]]
    elif np.array(vetor_resultado['S'])==3:
        resultado_markov= "Favorabilidade Alta"
        text+="\nAbordagem Cadeias Ocultas de Markov - Estado Atual:" + str(resultado_markov) + "\n\n"
        previsao_prox=[ [0], [0], [probab_previsao.sum()]]

    return previsao_prox, resultado_markov, text

# Chamada de Função para identificar o Estado do Atual da Favorabilidade FAS
#previsao_prox, resultado_markov, text=identificaEstadoAtual(vetor_resultado,\
#    probab_previsao, text)

In [ ]:
# Função para calcular Próximas Previsão das Probabilidades 
# Uso da Probabilidade Inicial
def calcularPrevisoes(matriz_transicao, probab_inicial):
    previsao_1=np.matmul(matriz_transicao, probab_inicial)
    print("Previsão 1:",previsao_1)

    # Previsão das Probabilidades - Uso da Probabilidade
    # -> Combinação da Tabela Verdade selecionada
    previsao_2=np.matmul(matriz_transicao, previsao_prox)
    print("Previsão 2:",previsao_2)

    return previsao_1, previsao_2

# Chamada de Função para calcular Próximas Previsão das Probabilidades
#previsao_1, previsao_2=calcularPrevisoes(matriz_transicao, probab_inicial)

In [156]:
# Script SQL para inserir dados na tabela "FAVORABILIDADES_FAS" no Banco Oracle (Cloud)
def gerarScriptSQLFavorabilidades_FAS(ID_BANCO_IMAGENS,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,JANELA_TEMPO_INICIAL,\
    JANELA_TEMPO_FINAL,ABORDAGEM_FUSAO_DADOS, vetor_compara,flag_abordagem):

    # Verificar a abordagem (flag_abordagem)
    # 1- Abordagem Figura de Mérito
    # 2- Abordagem Lógica Fuzzy
    # 3- Abordagem Cadeias Ocultas de Markov    

    if flag_abordagem==3:
        # Se Abordagem de Markov serão adicionados os valores (variável vetor_compara) para as colunas de "V1,V2,V3,V4,V5,V6,V7"
        # Inserir dados de inserção da tabela "FAVORABILIDADE_FAS"
        sql="insert into FAVORABILIDADES_FAS (ID_FAVORABILIDADE,ID_BANCO_IMAGENS,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,JANELA_TEMPO_INICIAL,"+\
        "JANELA_TEMPO_FINAL,ABORDAGEM_FUSAO_DADOS,V1,V2,V3,V4,V5,V6,V7)"
        # Inserir dados de valores da tupla
        sql_values="values (ID_FAVORABILIDADE_FAS_SEQ.nextval," + str(ID_BANCO_IMAGENS) + ",'" + str(RESULTADO_FAVORABILIDADE) +     \
            "','" + str(VETOR_FAVORABILIDADE) + "',TO_DATE(" + str(JANELA_TEMPO_INICIAL) +                                          \
            "),TO_DATE(" + str(JANELA_TEMPO_FINAL) + "),'" + str(ABORDAGEM_FUSAO_DADOS) + "',"+ str(vetor_compara[0])+ "," +        \
                str(vetor_compara[1])+ "," + str(vetor_compara[2])+ "," + str(vetor_compara[3]) + "," + str(vetor_compara[4]) + "," +\
                    str(vetor_compara[5])+ "," + str(vetor_compara[6]) + ");\n"

    else:
        # Se Abordagem diferente de Markov serão adicionados os valores = "null" para as colunas de "V1,V2,V3,V4,V5,V6,V7"
        # Inserir dados de inserção da tabela "FAVORABILIDADE_FAS"
        sql="insert into FAVORABILIDADES_FAS (ID_FAVORABILIDADE,ID_BANCO_IMAGENS,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,JANELA_TEMPO_INICIAL,"+\
        "JANELA_TEMPO_FINAL,ABORDAGEM_FUSAO_DADOS)"
        # Inserir dados de valores da tupla
        sql_values="values (ID_FAVORABILIDADE_FAS_SEQ.nextval," + str(ID_BANCO_IMAGENS) + ",'" + str(RESULTADO_FAVORABILIDADE) +     \
            "','" + str(VETOR_FAVORABILIDADE) + "',TO_DATE(" + str(JANELA_TEMPO_INICIAL) +                                          \
            "),TO_DATE(" + str(JANELA_TEMPO_FINAL) + "),'" + str(ABORDAGEM_FUSAO_DADOS) + "');\n"

    #incrementar uma linha do script
    text=''
    text+=sql + sql_values
       
    return text

# Script SQL para inserir dados na tabela "IMAGEM_CLIMA_FAVORAB" no Banco Oracle (Cloud)
def gerarScriptSQLImagemClima_Favorab(dataframe_dados,resultado_markov):
    text=''
    for idx, dados in dataframe_dados.iterrows():
        # Inserir dados de inserção da tabela "FAVORABILIDADE_FAS"
        sql="insert into IMAGEM_CLIMA_FAVORAB (ID_IMAGEM_CLIMA_FAVORAB,ID_BANCO_IMAGENS,ID_TAB_VERD_FAVORAB,ID_DADOS_CLIMATICOS,DESC_FAVORAB)"
        # Inserir dados de valores da tupla
        sql_values="values (ID_IMAGEM_CLIMA_FAVORAB_SEQ.nextval," + str(dados.ID_BANCO_IMAGENS) + ",'" + str(dados.ID_TAB_VERD_FAVORAB) + "','" + str(dados.ID_DADOS_CLIMATICOS) +\
            "','" + resultado_markov + "');\n"
        #incrementar uma linha do script
        text+=sql + sql_values
       
    return text

# Salvar script SQL em disco
def salvarScriptSQL(text,path_2,nome):
    data_hora = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    with open(path_2 + nome + str(data_hora) + '.sql', 'w') as f:
        f.write(text)

In [157]:
# # Chamada de Função Script SQL para inserir dados de Favorabilidade no Banco Oracle (Cloud)

# #------------------------------------------------------------------------
# #### Figura de Mérito
# #------------------------------------------------------------------------
# # Armazenar o Resultado de Favorabilidade= +5% Erro
# RESULTADO_FAVORABILIDADE=str(lista_resultado_figura[1]).replace("'","")
# VETOR_FAVORABILIDADE=str(lista_resultado_figura).replace("'","")
# ABORDAGEM_FUSAO_DADOS='Figura de Mérito'
# SQL_text=''
# SQL_text+=gerarScriptSQLFavorabilidades_FAS(1,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,DataInicial,DataFinal,ABORDAGEM_FUSAO_DADOS)

# #------------------------------------------------------------------------
# #### Lógica Fuzzy
# #------------------------------------------------------------------------
# # Armazenar o Resultado de Favorabilidade= +5% Erro
# RESULTADO_FAVORABILIDADE=str(lista_resultado_fuzzy[1]).replace("'","")
# VETOR_FAVORABILIDADE=str(lista_resultado_fuzzy).replace("'","")
# ABORDAGEM_FUSAO_DADOS='Lógica Fuzzy'
# SQL_text+=gerarScriptSQLFavorabilidades_FAS(1,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,DataInicial,DataFinal,ABORDAGEM_FUSAO_DADOS)

# #------------------------------------------------------------------------
# #### Cadeias Ocultas de Markov
# #------------------------------------------------------------------------
# # Armazenar o Resultado de Favorabilidade= +5% Erro
# RESULTADO_FAVORABILIDADE=resultado_markov
# labels_markov=['V1','V2','V3','V4','V5','V6','V7','S','P_V1','P_V2','P_V3','P_V4','P_V5','P_V6','P_V7']
# lista_resultado_markov=[]

# for index in zip(labels_markov,vetor_resultado.transpose().values.tolist()):
#     lista_resultado_markov.append(index)

# VETOR_FAVORABILIDADE=str(lista_resultado_markov).replace("'","")
# ABORDAGEM_FUSAO_DADOS='Cadeias Ocultas de Markov'
# SQL_text+=gerarScriptSQLFavorabilidades_FAS(1,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,DataInicial,DataFinal,ABORDAGEM_FUSAO_DADOS)

# # Chamada de Função para Salvar script SQL de todas as imagens processadas
# salvarScriptSQL(SQL_text,path_4,"SQL_Favorabilidade_")

# # Chamada de função para gerar o Script Sql - Dados da Tabela
# SQL_text=gerarScriptSQLImagemClima_Favorab(df_img_cli_favorab)

# # Chamada de Função para Salvar script SQL de todas as imagens processadas
# salvarScriptSQL(SQL_text,path_4,"SQLImagemClima_Favorab_")

In [158]:
# Função para gravar documento (formato .docx) contendo o relatório e imagens
def document_save_docx(text,path_out):
    document = Document()
    p = document.add_paragraph(text)
    r = p.add_run()
        
    r.add_picture(path_out + 'Figura_Merito.png',width=Inches(2.5), height=Inches(1.7))
    r.add_picture(path_out + 'Grafico_Fuzzy.png',width=Inches(2.0), height=Inches(1.5))
    #r.add_text('\n\n---------------------------------------------------------------------------------')
    
    ## Habilitar a linha abaixo para uso dos testes supervisionados
    ##document.save(path_out + 'Relatorio_'+ str(df_consulta_bd['dataMedicao'].values[0]).replace("/","-") + '.docx')
    
    document.save(path_out +'Relatorio_'+ str(DataInicial).replace("/","-").strip("'") + '.docx')

# Chamada de Função para gravar documento (formato .docx) contendo o relatório e imagens
#document_save_docx(text,path_3)

In [159]:
# # Chamada de Função para Registrar Colunas de Avaliação Automática no Relatório
# coluna_seq_logica, coluna_abordagens, coluna_figura_merito, coluna_logica_fuzzy, coluna_markov,                    \
#         coluna_favorabilidade_figura_Merito, coluna_favorabilidade_logica_fuzzy, coluna_favorabilidade_markov=     \
#         registrarResultadoAvaliacao(lista_resultado_figura, lista_resultado_fuzzy, resultado_markov, linha,        \
#         df_tabela, vetor_ocorrencias, vetor_resultado, coluna_abordagens, coluna_seq_logica, coluna_figura_merito, \
#         coluna_logica_fuzzy, coluna_markov)

In [160]:
#regras.gerarCombinacoes()

In [161]:
#Chamada de Função para Entrada de Data da Janela de Tempo
#Informar a Janela de Tempo como "Intervalo"

DataInicial, DataFinal = dataJanelaTempo(intervalo,0)

# Chamada de função para conexão com o Oracle Autonomous Database
connection=conexaoAutonomousDatabase(fun.PYTHON_USERNAME,fun.PYTHON_PASSWORD,fun.PYTHON_CONNECTSTRING)

###------------------------------------------------------------------------------------------###

# Chamadada de Função para consulta de Dados Climáticos (Janela de Tempo) no Banco ORACLE
df_consulta_bd=consultarDadosClimaticos(connection)

###------------------------------------------------------------------------------------------###

# Chamada de Função para gerar a lista de dados interpolados, de acordo com as Variáveis climáticas
X, lista_interpolacao, labels=preparaDadosInterpolacao(df_consulta_bd)

###------------------------------------------------------------------------------------------###

# Chamada de Função para Processamento de Interpolação
df_interpolar,new_y=processarInterpolacao(X,labels,lista_interpolacao)

###------------------------------------------------------------------------------------------###

# Chamada de Função para processamento dos valores interpolados para completar (se necessário) o conjunto de dados com os dados faltantes
df_consulta_bd=processarCompletarDadosInterpolados(df_interpolar,df_consulta_bd)

###------------------------------------------------------------------------------------------###

# Chamada de Função Regra 01 -> (Período de Molhamento Foliar)
medianaRegra_01, regra_1, flag_regra_1=regra01(df_consulta_bd,casas_decimais)

###------------------------------------------------------------------------------------------###

# Chamada de Função Regra 02 -> (Período Mínimo de Molhamento Foliar)
medianaRegra_02, regra_2, flag_regra_2=regra02(df_consulta_bd,casas_decimais)

###------------------------------------------------------------------------------------------###

#Chamada de Função Regra 03 
medianaRegra_03_1, medianaRegra_03_2, regra_3, flag_regra_3, flag_df=regra03(df_consulta_bd,casas_decimais)

###------------------------------------------------------------------------------------------###

# Chamada Função Regra 04
regra_4, regra_4_valor, flag_regra_4=regra04(df_consulta_bd,casas_decimais)

###------------------------------------------------------------------------------------------###

# Chamada de Função Regra 05
regra_5, regra_5_valor, flag_regra_5=regra05(df_consulta_bd,casas_decimais)

###------------------------------------------------------------------------------------------###

# Chamada de Função Regra 06
regra_6, medianaRegra_06, flag_regra_6=regra06(df_consulta_bd,casas_decimais)

###------------------------------------------------------------------------------------------###

# Chamada de Função Regra 07
regra_7, df_consulta_img, flag_regra_7, imagem_retornada=regra07(DataInicial,DataFinal)

###------------------------------------------------------------------------------------------###

#Chamada de Função para Exibir Imagens Segmentadas participantes da Janela de Tempo
#exibirImagensParticipantes(df_consulta_img)

###------------------------------------------------------------------------------------------###

# Função para Contagem de Ocorrências por Regras
dados_regras, totalJanela, ocorrenciasRegra_1, ocorrenciasRegra_2, ocorrenciasRegra_3, ocorrenciasRegra_4,                   \
ocorrenciasRegra_5, ocorrenciasRegra_6, ocorrenciasRegra_7=contagemOcorrencias(df_consulta_bd,regra_1,regra_2,regra_3,    \
regra_4,regra_5,regra_6,regra_7,medianaRegra_01,medianaRegra_02,regra_4_valor,regra_5_valor,medianaRegra_06, casas_decimais, flag_df)

###------------------------------------------------------------------------------------------###

# Chamada de Função para Plotar gráfico de ocorrências
x,y=plotarOcorrencias(ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,ocorrenciasRegra_4,ocorrenciasRegra_5,\
ocorrenciasRegra_6,ocorrenciasRegra_7)

###------------------------------------------------------------------------------------------###

#Chamada de Função para Plotar gráfico de ocorrências (Regra 1)
if not medianaRegra_01==0:
    z=plotarGraficoOcorrenciasRegra01(x,y,regra_1)
else:
    print("Não há dados para serem plotados para a Regra 1!!")
    
###------------------------------------------------------------------------------------------###

# Chamada de Função para Plotar gráfico de ocorrências (Regra 2)
if not medianaRegra_02==0:
    plotarGraficoOcorrenciasRegra02(x,y,regra_2)
else:
    print("Não há dados para serem plotados para a Regra 2!!")
    
###------------------------------------------------------------------------------------------###
    
# Chamada de Função para Plotar gráfico de ocorrências (Regra 3)
if type(regra_3)==int:
    if (regra_3==0):
        print("Não há dados para serem plotados para a Regra 3!!")

elif regra_3.empty:
    print("Não há dados para serem plotados para a Regra 3!!")

else:
    plotarGraficoOcorrenciasRegra03(x,y,regra_3)
    
###------------------------------------------------------------------------------------------###

# Chamada de Função para Plotar gráfico de ocorrências (Regra 5)
plotarGraficoOcorrenciasRegra05(x,y,regra_5)

###------------------------------------------------------------------------------------------###

### Chamadas de Função Abordagem: Figura de Mérito ###

# Chamada de função para definição dos ângulos para o processo
angulo_0,angulo_1,angulo_2,angulo_3,angulo_4,angulo_5,angulo_6 = definirAngulos(angulo)

###------------------------------------------------------------------------------------------###

# Chamada de função para desenhar a Figura de Mérito
c_6, c_7, c_8, c_9, c_10, c_11, c_12, coord_x_reta_0,coord_y_reta_0, coord_x_reta_1, coord_y_reta_1,        \
    coord_x_reta_2,coord_y_reta_2, coord_x_reta_3,coord_y_reta_3, coord_x_reta_4, coord_y_reta_4,           \
    coord_x_reta_5, coord_y_reta_5, coord_x_reta_6, coord_y_reta_6, valor_aproximado_ponto_0,               \
    valor_aproximado_ponto_1, valor_aproximado_ponto_2, valor_aproximado_ponto_3, valor_aproximado_ponto_4, \
    valor_aproximado_ponto_5, valor_aproximado_ponto_6 = plotUnitCircle(angulo_0,angulo_1,angulo_2,angulo_3,\
    angulo_4,angulo_5,angulo_6)

###------------------------------------------------------------------------------------------###

# Chamada de Função para Exibir os valores de entrada da Figura de Mérito
text=exibirEntradaFiguraMerito(text)

# Chamada de função para calcular os resultados da Figura de Mérito
lista_resultado_figura, lista_resultado, text=resultadosFiguraMerito(valor_aproximado_ponto_0, valor_aproximado_ponto_1, valor_aproximado_ponto_2,\
    valor_aproximado_ponto_3, valor_aproximado_ponto_4, valor_aproximado_ponto_5, valor_aproximado_ponto_6, angulo, text)
    
###------------------------------------------------------------------------------------------###
    
## Chamada de função para definição de Variáveis do Problema
periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade=definirVariaveisFuzzy()

###------------------------------------------------------------------------------------------###
        
# Chamada de função para criar as funções de pertinência tipo triângulo (mapeamento entre valores nítidos e difusos)
funçoesPertinencia(periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade)
        
###------------------------------------------------------------------------------------------###
        
# Chamada de Função para Visualização gráfica das Funções de Pertinência
visualizarFuncoesPertinencia(periodoMolhamentoFoliar, periodoMinMolhamentoFoliar, dadosCultivarImagem, pontoOrvalho, faixaTemperaturaInicial,\
        faixaTemperaturaFinal, temperaturaMinima, temperaturaMaxima, favorabilidade)

###------------------------------------------------------------------------------------------###

# Implementação das Regras Fuzzy

# Chamada de Função para definição das Regras Fuzzy 
rule1_1, rule1_2, rule1_3, rule1_4, rule1_5, rule1_6, rule1_7, rule1_8,\
        rule2_1, rule2_2, rule2_3, rule2_4, rule2_5, rule2_6, rule2_7, rule2_8, rule2_9, rule2_10,rule2_11, \
        rule2_12, rule2_13, rule2_14, rule2_15, rule2_16, rule2_17, rule2_18, rule2_19, rule2_20, rule2_21, \
        rule3_1,rule3_2,rule3_3,rule3_4,rule3_5,rule3_6,rule3_7,rule3_8,rule3_9,rule3_10,rule3_11,rule3_12, \
        rule3_13,rule3_14,rule3_15,rule3_16, rule3_17,rule3_18,rule3_19,rule3_20,rule3_21,rule3_22,rule3_23,\
        rule3_24,rule3_25,rule3_26,rule3_27,rule3_28,rule3_29,rule3_30,rule3_31, rule3_32,rule3_33,rule3_34,rule3_35, \
        rule4_1,rule4_2,rule4_3,rule4_4,rule4_5,rule4_6,rule4_7,rule4_8,rule4_9,rule4_10,rule4_11,rule4_12,rule4_13,  \
        rule4_14,rule4_15,rule4_16,rule4_17,rule4_18,rule4_19,rule4_20,rule4_21,rule4_22,rule4_23,rule4_24,rule4_25,  \
        rule4_26,rule4_27,rule4_28,rule4_29,rule4_30,rule4_31,rule4_32,rule4_33,rule4_34,rule4_35,\
        rule5_1, rule5_2, rule5_3, rule5_4, rule5_5,rule5_6, rule5_7, rule5_8, rule5_9, rule5_10,rule5_11, rule5_12,  \
        rule5_13, rule5_14, rule5_15,rule5_16, rule5_17, rule5_18, rule5_20,rule5_21= regras.regras(
        periodoMinMolhamentoFoliar,periodoMolhamentoFoliar,dadosCultivarImagem,pontoOrvalho,faixaTemperaturaInicial,
        faixaTemperaturaFinal,temperaturaMinima,temperaturaMaxima, favorabilidade)

###------------------------------------------------------------------------------------------###
        
# Chamada de Função para Simulação do Controlador Fuzzy
favorabilidade_ctrl=simularControladorFuzzy()

###------------------------------------------------------------------------------------------###

# Chamada de Função para Processar as regras no controlador Fuzzy
favorabilidade_simulador, resultado=processarRegrasFuzzy(favorabilidade_ctrl)

###------------------------------------------------------------------------------------------###

# Chamada de Função para o Cálculo dos Erros:
lista_resultado=calcularErro(resultado)

###------------------------------------------------------------------------------------------###

# Chamada de Função para Mostrar os Resultados Fuzzy
lista_resultado_fuzzy, text=mostrarResultadosFuzzy(lista_resultado,favorabilidade,favorabilidade_simulador,text)

###------------------------------------------------------------------------------------------###

# Chamada de Função para Carregar a Tabela Verdade (Abordagem Markov)
df_tab_verdade=carregarTabelaVerdade()

###------------------------------------------------------------------------------------------###

# Chamada de Função para encontrar a Combinação da Tabela - Descobrir o Estado correspondente
vetor_resultado, vetor_compara, vetor_ocorrencias, text, coluna_vetor_ocorrencias=encontraCombinacao(ocorrenciasRegra_1,ocorrenciasRegra_2,ocorrenciasRegra_3,\
    ocorrenciasRegra_4,ocorrenciasRegra_5,ocorrenciasRegra_6,ocorrenciasRegra_7, text, coluna_vetor_ocorrencias)
    
###------------------------------------------------------------------------------------------###
    
# Chamada de Função para capturar o índice do resultado do "vetor_resultado"
df_img_cli_favorab,text=capturarIndiceFavorab(vetor_resultado,text,imagem_retornada)

###------------------------------------------------------------------------------------------###

# Chamada de Função para identificar a probabilidade do Estado Atual
probab_estado, probab_previsao, matriz_transicao, probab_inicial, text=probabilidadeEstadoAtual(text)

###------------------------------------------------------------------------------------------###

# Chamada de Função para identificar o Estado do Atual da Favorabilidade FAS
previsao_prox, resultado_markov, text=identificaEstadoAtual(vetor_resultado,\
    probab_previsao, text)
    
###------------------------------------------------------------------------------------------###

# Chamada de Função para calcular Próximas Previsão das Probabilidades
previsao_1, previsao_2=calcularPrevisoes(matriz_transicao, probab_inicial)

###------------------------------------------------------------------------------------------###

# Chamada de Função Script SQL para inserir dados de Favorabilidade no Banco Oracle (Cloud)

#------------------------------------------------------------------------
#### Figura de Mérito
#------------------------------------------------------------------------
# Armazenar o Resultado de Favorabilidade= +5% Erro
RESULTADO_FAVORABILIDADE=str(lista_resultado_figura[1]).replace("'","")
VETOR_FAVORABILIDADE=str(lista_resultado_figura).replace("'","")
ABORDAGEM_FUSAO_DADOS='Figura de Mérito'
SQL_text=''
SQL_text+=gerarScriptSQLFavorabilidades_FAS(imagem_retornada,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,DataInicial,\
    DataFinal,ABORDAGEM_FUSAO_DADOS,vetor_compara,1)

#------------------------------------------------------------------------
#### Lógica Fuzzy
#------------------------------------------------------------------------
# Armazenar o Resultado de Favorabilidade= +5% Erro
RESULTADO_FAVORABILIDADE=str(lista_resultado_fuzzy[1]).replace("'","")
VETOR_FAVORABILIDADE=str(lista_resultado_fuzzy).replace("'","")
ABORDAGEM_FUSAO_DADOS='Lógica Fuzzy'
SQL_text+=gerarScriptSQLFavorabilidades_FAS(imagem_retornada,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,DataInicial,\
    DataFinal,ABORDAGEM_FUSAO_DADOS,vetor_compara,2)

#------------------------------------------------------------------------
#### Cadeias Ocultas de Markov
#------------------------------------------------------------------------
# Armazenar o Resultado de Favorabilidade= +5% Erro
RESULTADO_FAVORABILIDADE=resultado_markov
labels_markov=['V1','V2','V3','V4','V5','V6','V7','S','P_V1','P_V2','P_V3','P_V4','P_V5','P_V6','P_V7']
lista_resultado_markov=[]

for index in zip(labels_markov,vetor_resultado.transpose().values.tolist()):
    lista_resultado_markov.append(index)

VETOR_FAVORABILIDADE=str(lista_resultado_markov).replace("'","")
ABORDAGEM_FUSAO_DADOS='Cadeias Ocultas de Markov'
SQL_text+=gerarScriptSQLFavorabilidades_FAS(imagem_retornada,RESULTADO_FAVORABILIDADE,VETOR_FAVORABILIDADE,DataInicial,\
    DataFinal,ABORDAGEM_FUSAO_DADOS,vetor_compara,3)

# Chamada de Função para Salvar script SQL de todas as imagens processadas
salvarScriptSQL(SQL_text,path_4,"SQL_Favorabilidade_")

# Chamada de função para gerar o Script Sql - Dados da Tabela
SQL_text=gerarScriptSQLImagemClima_Favorab(df_img_cli_favorab,resultado_markov)

# Chamada de Função para Salvar script SQL de todas as imagens processadas
salvarScriptSQL(SQL_text,path_4,"SQLImagemClima_Favorab_")

###------------------------------------------------------------------------------------------###

# # Chamada de Função para gravar documento (formato .docx) contendo o relatório e imagens
document_save_docx(text,path_3)

###------------------------------------------------------------------------------------------###
